# Predicting Housing Prices, Exploratory Data Analysis
## Overview

The dataset is a Housing Prices dataset found [here](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview) 

We will build a regression model to predict housing prices and submit to the competition.

## Basic Exploration

We begin with a shallow investigation of the features. The goals are to:
1. Determine which features are numeric and which are categorical
2. Determine if any features are missing

Using the above information, we will perform the bare minimum data preprocessing to get a linear regression model working. This baseline model will be the benchmark we will improve our models against.


In [ ]:
!pip install -U scikit-learn
!pip install -U statsmodels
!pip show scikit-learn

     |████████████████████████████████| 22.3MB 1.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 9.5MB 3.5MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
Name: scikit-learn
Version: 0.24.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, scipy, joblib, threadpoolctl
Required-by: yellowbrick, textgenrnn, sklearn, sklearn-pandas, mlxtend, lightgbm, librosa, imbalanced-learn, fancyimpute


In [ ]:
# Mount so that we can access files on Google Drive
from google.colab import drive
drive.mount("/content/drive")

KeyboardInterrupt: ignored

In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/housing-prices/"

In [ ]:
# Loading the data
import pandas as pd

workdir = "/content/drive/My Drive/Colab Notebooks/housing-prices/"
train_filepath = workdir + "train.csv"
test_filepath = workdir + "test.csv"

raw_data = pd.read_csv(train_filepath, index_col="Id")
X_test = pd.read_csv(test_filepath, index_col="Id")

target_col = 'SalePrice'
X = raw_data.drop(target_col, axis=1)
y = raw_data[target_col]

In [ ]:
raw_data.head()

In [ ]:
X.dtypes

In [ ]:
X.columns

In [ ]:
# Determine then numerical features from the categorical features
mask = X.dtypes == 'object'
object_cols = list(mask[mask].index)
numerical_cols = list(set(X.columns) - set(object_cols))

In [ ]:
object_cols

In [ ]:
numerical_cols

In [ ]:
from sklearn.model_selection import train_test_split

global_random_state = 0
global_valid_size = 0.3

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=global_valid_size, random_state=global_random_state
)

# Creating combined dataset in-case it is needed for EDA later.
combined_train = pd.merge(X_train, y_train, left_index=True, right_index=True)
combined_valid = pd.merge(X_valid, y_valid, left_index=True, right_index=True)

In [ ]:
null_count = X_train.isnull().sum(axis=0)
with_null = null_count[null_count > 0].index
with_null_types = X_train[with_null.tolist()].dtypes

The above EDA suggests the following minimum preprocessing to get a model working:
1. Encoding categorical features (we will use one-hot encoding for the baseline)
2. Feature scaling numerical features (we will use standardization)
3. Imputing numerical features (we will use mean imputation for the baseline)
4. Imputing categorical features (we will use the most frequent value for the baseline) 

## Constructing a Baseline Model

### Baseline Linear Regression Model
1. Construct a simple model
2. Perform the bare minimum data preprocessing necessary for the regression
3. Evaluate the model
4. Plot the learning curve
5. Visualize the linear regression if possible.

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

numerical_transformer = Pipeline(
    steps=[('num_impute', SimpleImputer(strategy='mean')),
           ('standardize', StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[('cat_impute', SimpleImputer(strategy='most_frequent')),
           ('ord_encoding', OneHotEncoder(handle_unknown="ignore"))]
)

# Using a subset of the features because using all 79 features will cause an explosion in MSE
baseline_num_cols = ['YearBuilt', 'OverallQual', 'OverallCond', 'LotArea', '1stFlrSF', 
                     '2ndFlrSF', 'BedroomAbvGr', 'WoodDeckSF', 'OpenPorchSF']
baseline_cat_cols = ['LotFrontage', 'KitchenQual', 'Heating', 'RoofStyle', 'RoofMatl',
                     'BldgType', 'Neighborhood']

preprocessor = ColumnTransformer(
    transformers=[('num', numerical_transformer, baseline_num_cols),
                  ('cat', categorical_transformer, baseline_cat_cols)]
)

model = LinearRegression()

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('model', model)])

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error, make_scorer
import matplotlib.pyplot as plt
import numpy as np

def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=None, 
                        train_sizes=np.linspace(.1, 1.0, 5), scoring=None):
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, 
                                                            n_jobs=n_jobs,
                                                            train_sizes=train_sizes, 
                                                            scoring=scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1,
                     color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    plt.legend(loc="best")
    
    return train_scores, test_scores

def pretty_learning_curve(clf, X, y, scoring, train_sizes):
  
  plt.xlabel("Dataset Size")
  plt.ylabel("Score")
  train_scores, valid_scores = plot_learning_curve(clf, "Learning Curve", X, y, 
                                                   train_sizes=train_sizes, scoring=scoring)
  plt.show()

  for train_size, scores in zip(train_sizes, train_scores):
    print("Train score - size %3d - mean %10f - scores %60s" % 
          (train_size, np.mean(scores), str(scores)))

  print()

  for train_size, scores in zip(train_sizes, valid_scores):
    print("Valid score - size %3d - mean %10f - scores %60s" % 
          (train_size, np.mean(scores), str(scores)))

X_baseline = X[baseline_num_cols + baseline_cat_cols]
y_baseline = y

train_sizes = [100 * i + 1 for i in range(0, 12)]
pretty_learning_curve(pipe, X_baseline, y_baseline, 
                      scoring=make_scorer(mean_squared_error), 
                      train_sizes=train_sizes)

In [ ]:
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import mean_squared_error

def score_classifier(pipe, X, y, cv_num=10, n_repeats=5):
  cv_strategy = RepeatedKFold(n_splits=cv_num, n_repeats=n_repeats, 
                              random_state=global_random_state)
  scoring_strategy = make_scorer(mean_squared_error)
  score = cross_val_score(pipe, X, y, cv=cv_strategy, 
                          n_jobs=-1, scoring=scoring_strategy)
  return np.mean(score), score

class Evaluator():
  def fit(self, pipe, X, y):
    self.baseline = score_classifier(pipe, X, y)[0]
    return self

  def evaluate(self, pipe, X, y):
    return score_classifier(pipe, X, y)[0] - self.baseline

  def get_baseline(self):
    return self.baseline

evaluator = Evaluator()
evaluator.fit(pipe, X_baseline, y_baseline)

print("The baseline cross-validation score is: %f" % 
      evaluator.get_baseline()) 

In [ ]:
# # Uncomment for submission to Kaggle
# pipe.fit(X, y)
# baseline_preds = pipe.predict(X_test)

# output = pd.DataFrame({'Id': X_test.index, 'SalePrice': baseline_preds})
# output.to_csv(workdir + '/submission.csv', index=False)

### Results
Kaggle provides a sample submissions in `submission.csv` which has a score of 0.40613. 

The baseline model we submit has a score of about 0.17987 on Kaggle, so this is the target we will beat.

As a proxy for the Kaggle score, we will beat the cross-validation score of: 1248912305.301590, when working on this notebook

### Next Steps

The model is below the expected performance. Therefore, we will consider the following to improve model performance:

1. Check the assumptions of linear regression and manipulate the data to meet the assumptions if possible.
2. Try alternative models such as Lasso, ElasticNet, RidgeRegression, and linear SVM. 

There might be overfitting, as the validation score has a gap with the training score. Although graphically this gap appears small, the cross-validation score is infact twice that of the training score. Ideally, the two scores should be as close as possible. 

## Exploratory Data Analysis

Our goals are:
1. Check feature assumptions
2. Check model assumptions 
3. Determine the best kind of categorical encoding for each column
4. Determine a good imputation strategy for each column
5. Produce descriptive statistics and note any interest patterns.

### Feature Documentation

Below, we've surveyed all the features listed in `data_description.txt` obtained from [here](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data). We classify the type of feature, categorize them if they seem related, estimate the importance of the feature in predicting the target based on intuition and some domain research, and then suggest data preprocessing steps. 

| Variable        | Feature Type                             | Category                     | Expected Predictive Weight on Target | Description                                                  | Proposed Preprocessing                                       |
| --------------- | ---------------------------------------- | ---------------------------- | ------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| `SalePrice`     | Continuous                               | Target                       | N/A                                  | The property's sale price in dollars.                        | N/A                                                          |
| `MSSubClass`    | Nominal [15]                             | Building, Qualitative        | High                                 | The type of dwelling gives building layout information, such as the number of stories, or a duplex building vs. fmaily conversion. Intuitively, 2-story should be more expensive than 1-story, so this may play a large role in `SalePrice`` | Needs more sophsiticated categorical encoding                |
| `MSZoning`      | Nominal [8]                              | Neighborhood                 | High                                 | Distinguishes zones such as `Agriculture` vs. `Commercial` vs. `Industrial`, etc. More details [here](https://www.cityofames.org/government/departments-divisions-i-z/legal/city-of-ames-municipal-code/zoning-table-of-contents-municipal-code-chapter-29).<br />Certain areas might be more desirable than others, e.g, if `Industrial` zones allow manufacturing plants, then nearby residences might have lower property value due to the unappealing industrial centers nearby. | Needs more sophsiticated categorical encoding                |
| `LotFrontage`   | Continuous (I)                           | Lot, Quantitative            | Medium                               | The linear feet of street connected to property. It is better illustrated [here](https://stats.stackexchange.com/questions/189678/is-it-better-to-do-exploratory-data-analysis-on-the-training-dataset-only). <br /><br />This might not be a significant variable. Intuitively, the influence of `LotFrontage` might be "logarithmic" in sense that a large frontage is not very valuable, but a small frontage could be inconvenient. | Standardization                                              |
| `LotArea`       | Continuous (I)                           | Lot, Quantitative            | Medium                               | The lot size in square feet. <br />Lot area might raise costs due to the inherent value of the land, however, for a homebuyer, it might not be a priority. | Standardization, no feature engineering (already in square ft.),  `SalePrice` is possibly linear with this feature |
| `Street`        | Nominal [2]                              | Building, Qualitative        | Low                                  | Type of road access to property: Gravel or Paved. Unlikely to be a significant factor for buyers. | Binary encoding should suffice                               |
| `Alley`         | Nominal [3]                              | Building, Qualitative        | Low                                  | Type of alley access to property: Gravel, Paved, or NA. Unlikely to be a significant factor for buyters. | One-hot encoding, drop if insignificant                      |
| `LotShape`      | Nominal [4]                              | Lot, Qualitative             | Medium                               | See [here](https://www.gimme-shelter.com/lot-shape-50068/). Seems to interact with `LotArea` and `LotFrontage`. According to the article, having a high `LotArea` is meaningless if the `LotShape` is concentrated on the front lot, which is worth less than the back lot. However, the area concentrated on the front lot could be inferred by a combination of `LotArea` and `LotShape`, so `LotShape` might be redundant | One-hot encoding, possibly redundant with `LotFrontage` and `LotArea`? |
| `LandContour`   | Nominal [4]                              | Land, Qualitative            | Low                                  | Includes level, banked, hillside, or depression. Unless the contour is distorted to the point of impractically, this isn't something significant that people consider. | One-hot encoding                                             |
| `Utilities`     | Nominal [4]                              | Building, Qualitative        | High                                 | This is one of the first concerns when buying a house. This is probably a significant influence on the `SalePrice` | One-hot encoding                                             |
| `LotConfig`     | Nominal [5]                              | Lot, Qualitative             | Low                                  | Includes `Inside lot`, `Corner lot`, and `CulDSac`. This does not seem to be a major factor for a buyer's choice | One-hot encoding                                             |
| `LandSlope`     | Ordinal [3]                              | Land, Qualitative            | Medium                               | This might be a somewhat influential factor in `SalePrice`. A severe slope might be inconvenient and drive down prices. | One-hot encoding                                             |
| `Neighborhood`  | Nominal [25]                             | Neighborhood                 | High                                 | This might be a significant factor. A neighborhood implies a lot of information: the quality of public education in the area, the crime rate of the neighborhood, and distance to travel the work. Independently of what the other features are, buyers might have subjective bias for particular neighborhoods. | Needs more sophisticated feature engineering. There are too many categories for one-hot encoding to be practical. Perhaps use target encoding and rank them by average `SalePrice`? The `Neighborhood` might nonlinearly influence all of the other features. Individuals of different social status may gravitate to different neighborhoods. Consequently, expensive renovations might drive up `SalePrice` in wealthier neighborhoods, moreso than lower class neighborhoods, where buyers would focus on the necessities over luxuries. |
| `Condition1`    | Nominal [9]                              | Neighborhood                 | Medium                               | Proximity to railroads or density-packed streets might reduced property value due to noise pollution. Proximity to parks might increase property value, since the neighborhood is perceived to be desirable or appealing. | Needs more sophisticated categorical encoding                |
| `Condition1`    | Nominal [9]                              | Neighborhood                 | Medium                               | Same as above                                                | Needs more sophisticated categorical encoding                |
| `BldgType`      | Nominal [5]                              | Building, Qualitative        | High                                 | Includes types such as: "Single-family detached" or "Townhouse End Unit." The type of housing influence how much space is included, which will impact the price. | One-hot encoding                                             |
| `HouseStyle`    | Nominal [8]                              | Building, Qualitative        | High                                 | Includes the number of floors such as 1 story, 1.5 story, 2 story, and whether the additional stories are finished or not. | Needs more sophisticated categorical encoding                |
| `OverallQual`   | Ordinal [10]                             | Building, Qualitative        | High                                 | General equality of the house is probably an influential factor in `SalePrice`. No-one wants to live in a home of shoddy make. | Ordinal encoding, standardization, might be nonlinear with `SalePrice` |
| `YearBuilt`     | Continuous (I)                           | Building, Quantitative       | Medium                               | The age of a house might contribute to its `SalePrice` , since the quality may decrease over time, which may lead to decrease in `SalePrice`. | Standardization                                              |
| `YearRemodAdd`  | Continuous (I)                           | Building, Quantitative       | Medium                               | Remodel date (same as construction date if no remodeling or additions). This might offset any depreciation of prices due to an old `YearBuilt` | Standardization, perhaps some feature engineering with `YearBuilt` or dropping `YearBuilt` entirely? |
| `RoofStyle`     | Nominal [6]                              | Building, Qualitative        | High                                 | Type of the roof. Roof shape and material might play a large role, because they will determine what it is like to live in the house. For example, glass walls have poor insulation. The make and type of roof might similarly affect the quality of living in the house. | Needs more sophisticated categorical encoding                |
| `RoofMaterial`  | Nominal [8]                              | Building, Qualitative        | High                                 | Material of the roof. Same as above.                         | Needs more sophisticated categorical encoding                |
| `Exterior1st`   | Nominal [17]                             | Building, Qualitative        | High                                 | The exterior material of the house might strongly impact the `SalePrice`, because: (1) the material used contributes the house's visual appeal and may impact `SalePrice`, (2) the materials used may affect the quality of living in the house, e.g, one of the values is "Asbestos Shingles" and if asbestos material is damaged and becomes airborne, it carries significant health risks. | Needs more sophisticated categorical encoding                |
| `Exterior2st`   | Nominal [17]                             | Building, Qualitative        | High                                 | Same as above                                                | Needs more sophisticated categorical encoding, feature engineering (possibly add a boolean indicator that a second exterior exists?) |
| `MasVnrType`    | Nominal [5]                              | Building, Qualitative        | High?                                | Masonry veneer appears to refer to an additional outer wall, according to [here](https://en.wikipedia.org/wiki/Masonry_veneer). <br />This feature appears to have the same consequences as `Exterior1st` and `Exterior2st`. | One-hot encoding                                             |
| `MasVnrArea`    | Continuous (I)                           | Building, Quantitative       | Low                                  | The area of the veneer might not be a factor that buyers typically focus on. | Standardization,  `SalePrice` is possibly linear with this feature |
| `ExterQual`     | Ordinal                                  | Building, Qualitative        | High                                 | The quality of the material, intuitively, will contribute to the `SalePrice`. People will likely pay less for houses that are  lower quality. | Ordinal encoding, standardization, feature engineering (`SalePrice` might not be a linear funciton of `ExteralQual`) |
| `ExterCond`     | Ordinal                                  | Building, Qualitative        | High                                 | The quality of the material, intuitively, will contribute to the `SalePrice`. People will likely pay less for houses that are  lower quality. | Ordinal encoding, standardization, feature engineering (`SalePrice` might not be a linear funciton of `ExteralCond`) |
| `Foundation`    | Nominal [6]                              | Building, Qualitative        | Low                                  | The `Foundation` might objectively contribute the quality of living in the home, but this does not seem like something that a buyer typically considers because it is not visible, so because of the buyer's subjective perception, it does not influence the `SalePrice` | Needs more sophisticated categorical encoding                |
| `BsmtQual`      | Ordinal                                  | Basement, Qualitative        | Medium                               | A basement does not seem necessary. Therefore, it might be seen as a bonus for buyers. Therefore, `SalePrice` would not be strongly influenced by this feature, because a "luxury" like a basement wouldn't justify a significant increase in price. | Ordinal encoding, standardization, might be nonlinear with `SalePrice` |
| `BsmtCond`      | Ordinal                                  | Basement, Qualitative        | Medium                               | Same as above.                                               | Ordinal encoding, standardization, might be nonlinear with `SalePrice` |
| `BsmtExposure`  | Ordinal                                  | Basement, Qualitative        | Low                                  | Refers to "walkout" or garden level walls. Concretely, this refers to a door from the basement to outside the house. This seems like a very low-priority asset that would not significantly influence `SalePrice` | Ordinal encoding, standardization, consider dropping this feature |
| `BsmtFinType1`  | Nominal [7], could consider as Ordinal   | Basement, Qualitative        | Medium                               | Refers to the quality of the basement: good living quarters, average living quarters, etc. | Ordinal encoding, standardization, `SalePrice` may be a nonlinear function of this feature |
| `BsmtFinSF1`    | Continuous (I)                           | Basement, Quantitative       | Medium                               | The surface area of the basement. Intuitively, a larger basement would influence the `SalePrice` | Standardization, `SalePrice` is possibly linear with this feature |
| `BsmtFinType2`  | Nominal [7], could consider as Ordinal   | Basement, Qualitative        | Medium                               | Same as `BsmtFinType1`                                       | Ordinal encoding, standardization, `SalePrice` may be a nonlinear function of this feature, feature engineering (perhaps add a boolean indicating a second finish type exists?) |
| `BsmtFinSF2`    | Continuous (I)                           | Basement, Quantitative       | Medium                               | Same as `BsmtFinSF1`                                         | Standardization, `SalePrice` is possibly linear with this feature, (feature engineering; two possibilties: (1) combine `BsmtFinSF1` and `BsmtFinSF2` to produce new total sf, (2) the price per unit surface area might depend on `BsmtFinType` so account for the value of the finish type) |
| `TotalBsmtSF`   | Continuous (I)                           | Basement, Quantitative       | Medium                               | Same as `BsmtFinSF1`                                         | Standardization, `SalePrice` is possibly linear with this feature, consider replacing `BsmtFinSF1` and `BsmtFinSF2` with this feature. The `FinSF` features are multicollinear and redundant. |
| `Heating`       | Nominal [6]                              | Heating, Qualitative         | High                                 | The value of this feature depends on the climate of the city. The data comes from Ames, Iowa, US, which has a "humid continental climate" according to [Wikipedia](https://en.wikipedia.org/wiki/Iowa). The city of Des Moines is close to Iowa, so we can infer the monthly normal highs/lows are similar. The winter temperature is relatively comparable to Toronto, possibly more severe, so good heating would be desirable. Hence, the feature might have a high influence on `SalePrice` | Needs more sophisticated categorical encoding. We could perform ordinal encoding if we can rank the heating by some measure (consider target encoding). |
| `HeatingQC`     | Ordinal                                  | Heating, Qualitative         | High                                 | Based on the reasoning above, the quality of heating might be crucial. | Ordinal encoding, standardization, might be nonlinear with `SalePrice`, might need to be used in tandem with `Heating` |
| `CentralAir`    | Nominal [2]                              | AC, Qualitative              | High                                 | From [this](https://fischerheating.com/decentralized-vs-centralized-hvac-system/) article, there are a number of considerations:<br />1. The choice of an HVAC system depends on several factors, so `CentralAir` should be used in tandem with building surface area<br />2. A centralized HVAC can be aesthetically pleasing to buyers, because the machinery is hidden behind the building.<br />3. Centralized systems have a high up-front cost compared to decentralized systems.<br />4. A decentralized system might have a lower lifespan and might induce maintenance costs.<br />The exact effect of `CentralAir` on `SalePrice` is unclear, but it is very likely that `CentralAir` has some impact. | Binary encoding                                              |
| `Electrical`    | Nominal [5], some ordinal ordering       | Electrical, Qualitative      | High                                 | The quality of the electrical system is intuitively one of the first things a buyer will consider when buying a house. | One-hot encoding                                             |
| `1stFlrSF`      | Continuous (I)                           | Building, Main, Quantitative | High                                 | Greater first floor surface area might imply a greater amount of land used. The raw value of land might drive up `SalePrice` | Standardization, possibly linear with `SalePrice`            |
| `2ndFlrSF`      | Continuous (I)                           | Building, Main, Quantitative | High                                 | More surface area on the second floor implies additional costs to construct the house, driving up `SalePrice`. It is likely that `2ndFlrSF <= 1stFlrSF` so the raw value of the land will have been accounted for by `1stFlrSF`. This implies that the second floor might be priced at a different rate compared to the first floor. | Standardization, possibly linear with `SalePrice`, consider summing this with `1stFlrSF`. On the other hand, different floors might have different pricing rates. |
| `LowQualFinSF`  | Continuous (I)                           | Building, Main, Quantitative | High                                 | If the amount of low qualified surface feet is significant, then this might make a negative impression to buyers and drive the `SalePrice` | Standardization. Possibly a non-linear relationship with `SalePrice`? A small amount of low qualify finish might be forgivable, but a large patch might turn off buyers, driving down the `SalePrice` |
| `GrLivArea`     | Continuous (I)                           | Building, Main, Quantitative | High                                 | Above grade living area might drive up the price.            | Standardization.                                             |
| `BsmtFullBath`  | Discrete                                 | Building, Main, Quantitative | Low                                  | Additional bathrooms might be convenient, but might not justify a markup in `SalePrice`. Buyers will likely be looking for bathrooms that look OK. As suggested by [this](https://www.opendoor.com/w/blog/improvements-that-increase-home-value) website, it seems that bathrooms will have less of an impact on `SalePrice` than other renovations. | Standardization                                              |
| `BsmtHalfBath`  | Discrete                                 | Building, Main, Quantitative | Low                                  | Same reasoning as above, but according to the Opendoor website, a half bathroom might be worth even less when marking up `SalePrice` | Standardization                                              |
| `FullBath`      | Discrete                                 | Building, Main, Quantitative | Medium                               | Same reasoning as above. The quality matters a bit more than the number of bathrooms, but it is possible that a fancy bathroom won't lead to a significant markup in `SalePrice` | Standardization                                              |
| `HalfBath`      | Discrete                                 | Building, Main, Quantitative | Medium                               | Same reasoning as above                                      | Standardization                                              |
| `Bedroom`       | Discrete                                 | Building, Main, Quantitative | High                                 | According to [this](https://www.opendoor.com/w/blog/improvements-that-increase-home-value) source, adding a 3rd bedroom results in a good increase in sale value. A bedroom might be more valuable than a `Bedroom` which it comes to increasing `SalePrice` | Standardization                                              |
| `Kitchen`       | N/A                                      | Missing feature              | N/A                                  | This feature is not actually included with the `.csv`        | N/A                                                          |
| `KitchenQual`   | Ordinal                                  | Building, Main, Quantitative | High                                 | According to [this](https://www.opendoor.com/w/blog/improvements-that-increase-home-value) source, remodelling a kitchen results in a good increase in sale value. | Ordinal encoding, standardization, might be nonlinear with `SalePrice` |
| `TotRmsAbvGrd`  | Discrete                                 | Building, Main, Quantitative | High                                 | `TotRmsAbvGrid` might give some information about the overall quality of the property, as opposed to one or two rooms such as the bathroom. The rationale is that, individually, a high quality room might not significantly impact `SalePrice`, but their cumulative value could drive up prices. | Standardization                                              |
| `Functional`    | Ordinal                                  | Building, Main, Qualitative  | (Very?) High                         | Having a functioning home without major issues will likely play a significant role into `SalePrice`. Some minor deductions could seriously drive down `SalePrice`. Major deductions could cause steep declines in `SalePrice` | Imputation (the data description states to asssume `Typ` unless deductions warranted, this suggests the imputation strategy should use `Typ` for missing values) |
| `Fireplaces`    | Discrete                                 | Building, Main, Quantitative | Low                                  | This seems like a luxury feature that wouldn't justify a significant increase `SalePrice`. Having 1 fireplace might be nice, but having 2 fireplaces is unnecessary. | Standardization                                              |
| `FireplaceQu`   | Ordinal                                  | Building, Main, Quantitative | Low                                  | Same reasoning as above. Might not significantly drive up `SalePrice`. | Ordinal encoding, standardization, might be nonlinear with `SalePrice` |
| `GarageType`    | Nominal [6]                              | Garage, Qualitative          | Low                                  | Where the garage is positioned doesn't seem like it would matter at all. | One-hot encoding                                             |
| `GarageYrBuilt` | Continuous [i]                           | Garage, Quantitative         | Low                                  | The year in which the garage was built might not matter, unless it strongly influences the quality of the garage. | Standardization                                              |
| `GarageFinish`  | Nominal [4], could be considered ordinal | Garage, Qualitative          | Medium                               | A high quality garage might be somewhat valuable, because a garage can act as a workshop or play area. However, it might not be as valuable as the "main" rooms of a house such as the kitchen or bedroom. | Ordinal encoding, standadization, possibly nonlinear         |
| `GarageCars`    | Discrete                                 | Garage, Quantitative         | Medium                               | Same reasoning as `GarageArea`                               | Standardization, possibly redundant with `GarageArea`?       |
| `GarageArea`    | Continuous (I)                           | Garage, Quantitative         | Medium                               | A garage consumes a significant amount of space. If we count purely the land value, then this might have a noticeable impact on `SalePrice` | Standardization, possibly linear with `SalePrice`            |
| `GarageQual`    | Ordinal                                  | Garage, Qualitative          | Medium                               | Quality of the garage likely impacts the `SalePrice`         | Ordinal encoding, standardization, possibly nonlinear.       |
| `GarageCond`    | Ordinal                                  | Garage, Qualitative          | Medum                                | Condition of the garage likely impacts the `SalePrice`       | Ordinal encoding, standardization, possibly nonlinear.       |
| `PavedDrive`    | Nominal, can be interpreted as Ordinal   | Garage, Qualitative          | Low                                  | This does not seem to be a influential factor in a home's overall quality. | Ordinal encoding, standardization, possibly nonlinear.       |
| `WoodDeckSF`    | Continuous (I)                           | Outdoor, Quantitative        | Low                                  | A wood deck might have some influence, but does not seem to be factor. | Standardization                                              |
| `OpenPorchSF`   | Continuous (I)                           | Outdoor, Quantitative        | Medium                               | An open porch might be a somewhat valuable luxury feature, but it is unlikely that it contributes significantly to `SalePrice` | Standardization                                              |
| `EnclosedPorch` | Continuous (I)                           | Outdoor, Quantitative        | Medium                               | The enclosed porch area in squar e feet. Same reasoning as above. Might contribute slightly more to `SalePrice` due to the cost of the enclosure. | Standardization                                              |
| `3SsnPorch`     | Continuous (I)                           | Outdoor, Quantitative        | Medium                               | The enclosed porch area in squar e feet. Same reasoning as above. Might contribute slightly more to `SalePrice` due to the cost of the enclosure and furnishing. | Standardization                                              |
| `PoolArea`      | Continuous (I)                           | Pool, Quantitative           | Medium                               | Ames weather will have extreme cold, so a pool is not always valuable. However, the raw cost of a pool might nonetheless drive up the `SalePrice` | Standardization                                              |
| `PoolQC`        | Ordinal                                  | Pool, Qualitative            | Medium                               | Pool quality is critical if a pool is under consideration by the buyer. A poorly maintained pool could be a hazard and would contribute negatively to `SalePrice` | Ordinal encoding, standardization, nonlinearity, should be used in tan dem with `PoolArea` |
| `Fence`         | Ordinal                                  | Main, Qualitative            | High                                 | The fence quality might be an important factor in the `SalePrice`. This asset might be closer to a necessity. | Ordinal encoding, standardization, nonlinearity.             |
| `MiscFeature`   | Ordinal                                  | Misc, Qualitative |     Medium | Includes luxury features like a tennis court or elevator, which may significantly drive up prices | 
| `MiscVal`       | Continuous (I)                           | Misc, Quantitative           | Medium                                    | Luxury features might drive up `SalePrice` | Standardization                                              |
| `MoSold`        | Discrete                                 | Sale, Quantitative           | High                                 | This could a meta-feature that nonlinearly influences other features. Financial bubbles, for example, might change the behaviour of buyers, and therefore, the expected influence of other features on `SalePrice` | Standardization; Meta-feature                                |
| `YrSold`        | Continuous (I)                           | Sale, Quantitative           | High                                 | Same as above                                                | Standardization; Meta-feature. Possibly combine this with `MoSold` somehow. |
| `SaleType`      | Nominal [10]                             | Sale, Qualitative            | High                                 | The type of sale might be influential in `SalePrice` depending on how `SalePrice` is recorded. Assuming `SalePrice` is the upfront payment, then a pure cash sale will report a higher `SalePrice`, than one that has significant interest. | Sophisticated categorical feature engineering;  meta-feature |
| `SaleCondition` | Nominal [6]                              | Sale, Qualitative            | High                                 | This might be a significant factor in `SalePrice`. One of the listed values is `Family`. It is possible that a sale between family members is more "forgiving" and thus will nonlinearly influence the other features, causing a lower `SalePrice` | Sophisticated categorical feature engineering;  Meta-feature; Feature engineering (features such as `SaleCondition` might have a nonlinear impact on the `SalePrice`, in the sense that, if all other features, $x$, contributes $cx$ to the `SalePrice`, then `SaleCondition` could be a feature that adjusts the value of the rate $c$, which is a nonlinear relationship). The rationale for this is that real-estate agents would want to maximize profit, so the rates $c$ might be driven up, whereas family members might offer less than market pricing out of generosity. |

(I) - Technically discrete integers, but effectively continuous for our purposes. 

- The expected predictive weight for each feature ranges from None, Low, Medium, or Strong.
- Generally speaking:
  - Necessities like utilities, bedrooms, kitchens, overall quality are probably High
  - Luxuries like porches, pools, fancy bathrooms, are probably Medium
  - Minor details like the pavement of the driveway are probaby Low.
  - Totally irrelevant information is None.

Note that the feature documentation is **inaccurate**. There are inconsistencies with what features are present in the dataset and what features are present in the documentation.

In [ ]:
# The features listed in `data_description.txt`
data_documentation_features = ['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'Bedroom', 'Kitchen', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition']

# Features in the actual dataset
X_features = list(X.columns)

In [ ]:
set(X_features) - set(data_documentation_features)

Hence, the dataset has the features `BedroomAbvGr` and `KitchenAbvGr` which are not documented. However, since `TotalRmsAbvGrd` is documented and means "total rooms above grade", we can infer that `BedroomAbvGr` and `KitchenAbvGr` means bedroom/kitchen above grade, respectively.

In [ ]:
set(data_documentation_features) - set(X_features)

We see that the data documentation describes features called `"Bedroom"` and `"Kitchen"`. The exact meaning of this feature is somewhat confusing, because the Kaggle description describes the feature with the phrase "above basement level" whereas the `data_description.txt` describes the feature with "above grade."

This [article](https://www.gimme-shelter.com/below-grade-50067/) suggests that "above grade" and "above basement level" mean the same thing.

This shows that `"Bedroom"` and `"Kitchen"` are equivalent to `"BedroomAbvGr"` and `"KitchenAbvGr"`, respectively. 

### Univariate Analysis



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import scoreatpercentile

#### Target

In [ ]:
y_train.describe()

The mean is ~180,000 which is greater than the median which is ~160,000. This suggests a right skewed distribution. 

50% of the data lies within the range [130000, 215000]

In [ ]:
sns.displot(y_train, kde=True)

In [ ]:
print("Skew: %f" % y_train.skew())
print("Kurtosis: %f" % y_train.kurt())

In [ ]:
# Normal distribution kurtosis and skew for reference.
ex = pd.Series(np.random.normal(size=10000))
print("Normal skew: %f" % ex.skew())
print("Normal kurtosis: %f" % ex.kurt())

The relative frequency histogram illustrates the right-skew of the `SalePrice`. The skewness and kurtosis further suggests that `SalePrice` does not follow a Normal distribution.

In [ ]:
sns.boxplot(y_train)

From the boxplot, there appear to be a significant number of outliers which have a higher price than expected.

In [ ]:
q75 = scoreatpercentile(y_train, 75)
q25 = scoreatpercentile(y_train, 25)
IQR = q75 - q25
outlier_threshold = q75 + 1.5 * IQR
outlier_threshold

Hence, non-outliers must have a `SalePrice` strictly less than the `outlier_threshold` above 

In [ ]:
nonoutliers = (y_train < outlier_threshold).sum()
outliers = (y_train >= outlier_threshold).sum()

print("Non-outliers: %d" % nonoutliers)
print("Outliers: %d" % outliers)
print("Outliers as percent of overall data: %f" % (outliers / len(y_train)))

Therefore, slightly less than 4% of the data are outliers. 

- **[ENG]** - Try removing the records correspond to `SalePrice` outliers to see if that improves model performance.
- **[TASK]** - Identify any interesting characteristics about the outliers.

#### Predictors

##### Summary Statistics

In [ ]:
X_train[numerical_cols].describe()

First, we define "luxury" to mean any asset of a house that is uncommon, say 75% of houses do not have it. We define "high-end" to mean any asset where the only 75th percentile value or greater indicates a presence of this asset. We define "average" to mean assets where the 50th percentile value or greater indicates a presence of this asset. 

Any feature marked with these labels needs further investigation to confirm if this is the case, and to check if these represent outliers, i.e, lavish, expensive homes. 

We make the following observations

- `2ndFlrSF` - At least 50% of the records have a value of 0 for `2ndFlrSF`. This indicates that having a second floor is not common.
- `KitchenAbvGr` - Since the 25th and 75th percentiles have value 1.0. This indicates that most people ($\ge 50$%) have only 1 kitchen above grade.
- `3SsnPorch` - Since the 75th percentile has value 0. This indicates that a significant number of houses do not have a 3-seasons porch. This asset is probably a luxury. 
- `BedroomAbvGr` - Since the 25th percentile has value 0, most houses have at least 2 bedrooms "above grade." A notable number of the population have at least 3 ($\ge 25$%). 
- `MSSubClass` - We categorized this as a nominal variable in the feature documentation. This is incorrectly typed.
- `YearRemodAdd` - Most of the housing ($\ge 50$%) were built in 1967-2003. 
- `LotFrontage` - There are few homes missing a `LotFrontage`. There are two possibilities: the value is non-zero and simply missing, the value should be zero because technically no street is connected to the lot. In reality, there could be a mix of both possibilities and other interpretations.
- `GarageCars` - Since the 25th percentile is 1.0, then 75% of homes have space for at least 1 car, 50% of homes have space for at least 2. 
- `OpenPorchSF` - At least 50% of homes do not have an open porch. 
- `LowQualFinSF` - At least 75% of homes do not have low quality finishes. We can infer from a box-and-whisker plot that any homes with `LowQualFinSF` are outliers
- `FullBath` - At least 75% of homes have a full bathroom, at least 50% have 2 full bathrooms. 
- `EnclosedPorch` - At least 75% of homes do not have an enclosed porch. This asset must be a luxury.
- `Fireplaces` - At least 50% of homes have a fireplace. This asset might not be as much of a luxury as previously assumed.
- `GarageYrBlt` - This has some missing values and requires deeper investigation to determine a proper imputation. 
- `BsmtUnfSF` - At least 75% of homes have at least 230 square feet of unfinished basement surface area. 
- `BsmtFinSF2` - At least 75% of homes do not have a second type of basement. We know then that houses with a second type of basement are outliers, possibly luxury homes.
- `GarageArea` - At least 25% of homes have at least 325 square feet of garage area.
- `OverallQual` - At least 25% of houses have at least material and finish quality of 5/10. At least 75% of more houses have quality at most 7/10. 
- `OverallCond` - At least 25% of houses have at least an overall condition of 5/10. At least 75% of more houses have condition at most 6/10. This and `OverallQual` is actually rather unsurprising because 5 indicates "Average" in the dataset's scale. 
- `GrLivArea` - 50% of homes have above-grade living area from around 1140 to 1790 square feet.
- `BsmtHalfBath` - At least 75% of people do not have a half-bathroom in the basement. This indicates that house that do have one are outliers. This asset is possibly a luxury. 
- `MiscVal` - At least 75% of homes do not have this miscellaneous value. Whatever it may be. This asset is possibly a luxury. 
- `LotArea` - At least 25% of homes have a lot area of atleast 7742.5 square feet.
- `PoolArea` - At least 75% of homes do not have any pool. This asset seems like a luxury feature. 
- `ScreenPorch` - At least 50% of homes do not have a screen porch. This asset appears to be a luxury feature. 
- `MasVnrArea` - At least 50% of homes do not have any masonry veneer. There are some missing values for this feature. This asset might be a high-end feature.
- `WoodDeckSF` - At least 50% of homes do not have a wood deck. This asset might be a high-end feature.
- `HalfBath` - This is a high-end feature.
- `MoSold` - This feature might be better visualized through a barplot.
- `BsmtFullBath` - High-end feature
- `BsmtFinSF` - Average feature. 
- `YearBuilt` - 1954-2000 is the construction date for 50% of homes recorded. It may be worthwhile computing the time between the `YearBuilt` and the year sold. 

The median or "average" home has the following characteristics:

- No second floor
- A total basement surface area of ~992 square feet.
- 1 kitchen above grade
- 3 bedrooms above grade
- A first floor surface area of ~1391 square feet.
- Remodelled in 1993 
- Lot frontage of 69 ft
- A garage for 2 cars
- An open porch of 25 square ft
- 0 square ft of low quality finishes. 
- 2 full bathrooms
- 1 fireplace
- 487 square ft of unfinished basement.
- 477 square ft of garage area 
- 1479 square ft of above basement living area. 
- 9536 square ft of lot area
- 6 total rooms above the basement
- 386 square ft of finished basement
- Built in 1972
- Bought in June 2008
- Lacks all high-end and luxury assets such as a 3 season porch or pool. 


We propose the following steps
- **[ENG]** - Try two feature engineering steps. Have `LotFrontage` be imputed to (1) 0 or (2) median (3) or mean. The choice of median vs. mean depends on the skew of the distribution, but it doesn't hurt to empirically verify. 
- **[ENG]** - Investigate `GarageYrBlt` for a good imputation value. This is either: (1) the same as the year the house itself was built or (2) not available because no garage was built. In the latter case, more sophisticated feature engineering is required. 
- **[TASK]** - Investigate all high-end and luxury features more closely.
- **[ENG]** - Try computing the number of years in-between `YearBuilt` and `YearSold` as a feature. 

In [ ]:
X_train[object_cols].describe()

We make the following observations: 
- `MSZoning` - Most housing is located in "Residential Low Density" zones
- `Street` - The road access to the property is predominantly paved. The classes are highly skewed.
- `Alley` - Many values are missing. Non-missing values appear to be balanced between paved and gravel roads.
- `LandContour` - Most housing is on level land. Unusual land shapes like banked, hillside, and depression are uncommon.
- `Utilities` - Nearly all housing has all public utilities. This feature might be useless for determining `SalePrice`, because there is only one record that lacks all public utilities.
- `LotConfig` - Most housing has "Inside lot" configuration.
- `LandSlope` - Most housing has gentle sloping. Any other form of sloping is uncommon. It's plausible that housing with severe sloping has low `SalePrice` because this quality is undesirable.
- `Condition1` - Most housing has Normal conditions. They are not near adverse features like noisy railroads or positive features like parks.
- `Condition2` - Almost all housing has Normal conditions for the second condition. One way to explain the higher proportion for Normal conditions compared to `Condition1` is if `Normal` for `Condition2` means there is no second special condition near the house. Hence, almost all housing is near at most 1 special condition.
- `BldgType` - Housing is predominantly single-family detached. 
- `HouseStyle` - About half of all housing has 1 story. The other half have additional stories.
- `RoofStyle` - Most housing has "gable" roof which is is essentially the "typical" triangular shape of a roof.
- `RoofMatl` - Almost all housing has "Standard (Composite) single as the roofing material. 
- `Qual`, `Cond` - Most quality or cond features have some value denoting "average" which seems unsurprising. This indicates that exceptionally good or bad housing is not the majority.
- `Heating` - Most heating uses gas. 
- `CentralAir` - Most housing uses centralized AC. This makes sense since this is likely to be the "default" AC system, with decentralized AC being a niche choice to fit some particular lifestyle, such as an environmentally-friendly one.
- `Electrical` - Most housing uses "Standard Circuit Breakers & Romex" 
- `Functional` - Most housing has typical functionality (no damages). 
- `PavedDrive` - Most housing a paved driveway
- `Fence` - If a house does have a fence, then it typically has minimum privacy. 
- `MiscFeature` - Most miscellaneous features, if they exist, are a simple shed.
- `SaleType` - Most are normal warranty deeds.
- `SaleCondition` - Most are normal sales.

Many of these categorical features might fit better in a decision tree. The impact of these variables often boils down to decision-making similar to the following: "A house with a less than typical value for `Functional` will have a severely low price due to damages driving down the property value." 

This kind of information can be encoded in linear regression using one-hot encoding, but it may overfit due to the presence of many features. One-hot encoding will also introduce many multicollinear features, as well, which is undesirable in linear regression.


In [ ]:
# Tracking down which columns were incorrectly typed when they were loaded
wrong_num_cols = ['MSSubClass']

##### Missing Values

In [ ]:
nc = null_count[null_count > 0]
nc

In [ ]:
nc_prop = nc / X_train.shape[0]
nc_prop.sort_values(ascending=False)

We make the following observations:
1. `PoolQC`, `MiscFeature`, `Alley`, `Fence` all have an excessive number of missing values. 
2. `FireplaceQu` nearly has half of its values missing. However, it may be reasonable to impute the 46% missing values, based on the 54% of data without missing values.
3. The remaining features have almost negligible numbers of missing features. 
4. Based on the overview of the feature documentation we performed earlier, the data collector deliberately put in a NA value for some of the columns. Rather than trying to "remove" the NA values by imputing them or dropping columns, we may want to consider the NAs as their own separate category.  

- **[TASK]**: Investigate `PoolQC`, `MiscFeature`, `Alley`, and `Fence` to determine why they tend to have missing values. Determine a viable imputation value or otherwise decide to discard these features.
- **[TASK]**: For `FireplaceQu` and all of the other features determine a viable imputation value. Preserve these features if possible, since they might have useful information. 

In [ ]:
# Investigate the nature of the missing values in each univariate feature. It might be worth target encoding NA values in ordinal variables.
with_missing = nc.index.to_list()

# Print out the numerical columns that have missing values
set(numerical_cols).intersection(with_missing)

In [ ]:
# Print out the object columns that have missing vlaues
set(object_cols).intersection(with_missing)

###### GarageYrBlt

From the above analysis, we see that `GarageYrBlt` is missing. This is problematic. The data description describes this feature as the year in which a garage is built. This implies that `GarageYrBlt` is missing if and only if `GarageArea` is 0. The code cell blelow confirms that this is the case.

To handle missing values, we could try imputing `GarageYrBlt`, but this does not make any business sense. Essentially, imputation will enforce the assumption that every house must a garage, which is factually incorrect. 

Fortunately, our later analysis on the correlation between numerical variables shows that `GarageYrBlt` is highly collinear with `YearBuilt`. Moreover, since `GarageYrBlt` being null and `GarageArea == 0` are equivalent facts, we can just drop `GarageYrBlt` to handle the missing values.

In [ ]:
_null_mask = combined_train['GarageYrBlt'].isnull()
_zero_mask = combined_train['GarageArea'] == 0

_with_null = combined_train[_null_mask]
_with_zero = combined_train[_zero_mask]
_with_null_and_zero = _with_null[_with_null['GarageArea'] == 0]

print("Total number of records with null GarageYrBlt or no GarageArea: %d" % (len(_with_null_and_zero)))
print("Total number of records with null GarageYrBlt: %d" % (len(_with_null)))
print("Total number of records with no GarageArea: %d" % (len(_with_zero)))

combined_train[_null_mask][['GarageYrBlt', 'GarageArea']]

###### Lot Frontage

We also see that `LotFrontage` is missing. The data description describes this as "linear feet of street connected to property". Ideally, NaN should indicate that there is 0 linear feet of street connected to property.  

The code cell below suggests that this might be the case.

In [ ]:
print(len(combined_train[combined_train['LotFrontage'] == 0]))
print(len(combined_train[combined_train['LotFrontage'].isnull()]))
print(len(combined_train[combined_train['LotFrontage'] > 0]))
print(len(combined_train))

We compute summary statistics for houses that have a `LotFrontage` value vs. houses that have a missing `LotFrontage` value to characterize houses falling into the latter category. 

In [ ]:
combined_train[combined_train['LotFrontage'].isnull()][numerical_cols + [target_col]].describe()

In [ ]:
combined_train[~(combined_train['LotFrontage'].isnull())][numerical_cols + [target_col]].describe()

From the above summary statistics for the numerical columns, we see that:
- `WoodDeckSF` - Seems to be slightly higher for houses lacking `LotFrontage`
- `PoolArea` - Appears to be slightly higher, but there are very data points that have a non-zero `PoolArea`, so the differences might be meaningless. 
- `LotArea` - Seems to be slighty higher for properties without `LotFrontage` 
- `BsmtFinSF2` - Appears to be somewhat higher for properties without `LotFrontage`
- `BsmtFinSF1` - Appears to be slightly lower for properties without `LotFrontage`
- `YearBuilt` - Properties without `LotFrontage` are built a few years later 
- `Fireplaces` - Slightly higher for properties without `LotFrontage` 
- `MiscVal` - Significantly higher for properties without `LotFrontage`. However, note that few properties have a miscellaneous asset, so this feature value might be distorted by outliers. 
- `LowQualFinSF` - Somewhat lower for properties without `LotFrontage` 

There do not seem to be particularly notable differences between the two in the numeric columns. Although `MiscVal` is higher, this is may have been distorted due to outliers.

In [ ]:
_no_frontage_object_cols = combined_train[combined_train['LotFrontage'].isnull()][object_cols]
_no_frontage_object_cols.describe()

In [ ]:
_yes_frontage_object_cols = combined_train[~(combined_train['LotFrontage'].isnull())][object_cols]
_yes_frontage_object_cols.describe()

We observe the following differences:

- `LotShape` - The houses without a `LotFrontage` are dominated by slightly irregular lots, whereas those with `LotFrontage` are more likely to have regular lots. 
- `Foundation` - Houses without a `LotFrontage` tend to have a cinderblock foundation, whereas for houses with a `LotFrontage`, a concrete foundation has the highest proportion of members (though it does not make up more than half of the observations).
- `BsmtQual` - More than half of the basements have good quality for houses without a `LotFrontage`. From the analysis of numerical columns, houses without a `LotFrontage` tend to have less unfinished garage area. 
- `BsmtFinType` - Houses with no `LotFrontage` tend to have "good living quarters" as the finish type with the highest members. House with `LotFrontage` have no basement finish as the dominant finish type.
- `FireplaceQu` - Houses with no `LotFrontage` tend to have average quality fireplaces, in contrast to good quality. 

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
sns.countplot(ax=ax[0], data=_no_frontage_object_cols, x="LotShape")
sns.countplot(ax=ax[1], data=_yes_frontage_object_cols, x="LotShape")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
sns.countplot(ax=ax[0], data=_no_frontage_object_cols, x="Foundation")
sns.countplot(ax=ax[1], data=_yes_frontage_object_cols, x="Foundation")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
sns.countplot(ax=ax[0], data=_no_frontage_object_cols, x="GarageType")
sns.countplot(ax=ax[1], data=_yes_frontage_object_cols, x="GarageType")

The difference between houses with and without `LotFrontage` is unclear. We cannot find a systematic reason why `LotFrontage` is missing. 

Hence, it is not obvious what the correct imputation value for `LotFrontage` is. An NA value could denote 0 lot frontage or that the non-zero lot frontage is not recorded. We will defer to experiments that empirically check which imputation method offers the best performance. 

- **[ENG]** - Try imputing `LotFrontage` with the mean, median, constant 0, or dropping the feature entirely. 

###### MasVnrArea

In the code cell below, we see for `MasVnrArea` that NA does not denote an area of 0, since there are records with `MasVnrArea == 0`. 

In [ ]:
_equals_zero = combined_train[combined_train['MasVnrArea'] == 0]
_gt_zero = combined_train[combined_train['MasVnrArea'] > 0]
_na = combined_train[combined_train['MasVnrArea'].isnull()]

print("MasVnrArea == 0: %d"           % len(_equals_zero))
print("MasVnrArea not available: %d"  % len(_na))
print("MasVnrArea > 0: %d"            % len(_gt_zero))
print("Total number of records: %d "  % len(combined_train))
print("%d + %d + %d = %d"             % (len(_equals_zero), 
                                         len(_na), 
                                         len(_gt_zero), 
                                         len(_equals_zero) + len(_na) + len(_gt_zero)))

We check the corresponding `MasVnrType` to better understand why `MasVnrArea` is missing. We see that either feature is missing if and only if the other is missing as well.

In [ ]:
print("The corresponding veneer type of records without masonry veneer area")
_na[['MasVnrArea', 'MasVnrType']]

In [ ]:
print("The number of records without a masonry veneer type: %d" %
      len(combined_train[combined_train['MasVnrType'].isnull()]))

Thus, the `MasVnrType` appears to be missing. This is interesting, because there are records where `MasVnrArea == 0`. The next question is what is the `MasVnrType` when `MasVnrArea == 0`? We show a sample of these records below. This demonstrates that, roughly speaking, `MasVnrType == None` if and only if `MasVnrArea == 0`. 

There is however, one outlier where the `MasVnrType` is `Stone` but the `MasVnrArea` is recorded as 0. This record appears to be incorrect.

In [ ]:
_equals_zero[['MasVnrArea', 'MasVnrType']]

In [ ]:
print(np.unique(_equals_zero['MasVnrArea']))
print(np.unique(_equals_zero['MasVnrType']))

In [ ]:
_equals_zero[_equals_zero['MasVnrType'] == 'Stone']

Compared to the average house illustrated by the summary statistics, the outlier house with id 1242 has
1. `BsmtUnfSF` - A high value of 1689
2. `1stFlrSF` - A high value of 1689
3. `SalePrice` - Significantly higher than the mean or median. 
4. `SaleCondition` - This house is being sold as `Partial`

According to the data documentation, we see that "Partial" means that "Home was not completed when last assessed (associated with New Homes)". This would explain the discrepancy, as the house might have been planned to have stone masonry, but when the house was assessed, the construction of the masonry was not started. 

For simplicity, we can drop this outlier.

Finally, we try to find any systematic reason for why `MasVnrArea` is missing. 

In [ ]:
_na.describe()

In [ ]:
_na[object_cols].describe()

If you compare the above summary statistics above with the summary statistics for the overall training set, there does not appear to be any noteworthy difference. 

We attempt two strategies for feature engineering: (1) impute `MasVnrArea` using the median/mean and `MasVnrType` with the most frequent observation, (2) first impute `MasVnrType` with the most frequent observation, then impute `MasVnrArea` with the average `MasVnrArea` for the most frequent `MasVnrType`.

The second approach might offer better accuracy, but we will empirically verify this.

- **[ENG]** - Try the two feature engineering strategies above.

###### Categorical Features

According to the data documentation, the following categorical features use "NA" to denote a known absence of some feature like alley-way access, rather than unknown information. We can replace "NA" in the `DataFrame` with `"Absent"` so that this NA category is accounted for by computations, such as  Cramer's V, which used in the categorical to categorical correlation matrix later in this notebook.

- `Alley` - NA means no alley access
- `BsmtCond` - NA means no basement
- `BsmtExposure` - NA means no basement
- `BsmtFinType1` - NA means no basement
- `BsmtFinType2` - NA means no basement
- `BsmtQual` - NA means no basement
- `Fence` - NA means no fence
- `FireplaceQu` - NA means no fireplace
- `GarageCond` - NA means no garage
- `GarageFinish` - NA means no garage
- `GarageQual` - NA means no garage
- `GarageType` - NA means no garage
- `MiscFeature` - NA means no misc feature
- `PoolQC` - NA means no pool

The following features use "NA" to denote unknown information, and thus, these values require feature engineering. 
- `Electrical` - Seems acceptable to impute using most frequent. 
- `MasVnrType` - Also discussed earlier in the `MasVnrArea` section


- **[ENG]** - Replace "NA" with its own category for the features in the first list.

In [ ]:
NA_category = 'Absent'
cols_with_NA_category = ['Alley', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
                         'BsmtFinType2', 'BsmtQual', 'Fence', 'FireplaceQu', 
                         'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType', 
                         'MiscFeature', 'PoolQC']


combined_train_with_absent = combined_train.copy()
combined_train_with_absent[cols_with_NA_category] = combined_train[cols_with_NA_category].fillna(NA_category)
combined_train_with_absent

###### Electrical

According to the data documentation, there should be no house that has the category `NA`. For simplicity, we will try to impute using two strategies: (1) using most frequent, (2) using "Absent"

- **[ENG]** - Impute the category for `Electrical`

##### Distributions for Numeric Variables

In [ ]:
# Plots all features in `cols` on a `nrows x ncols` grid. Each feature is
# graphed in the specified `plot` type.
def plot_grid(plot, cols, nrows=3, ncols=3, figlen=5):
  fig, ax = plt.subplots(nrows=nrows, ncols=ncols, 
                         figsize=(figlen * ncols, figlen * nrows))
  
  for k, feat in enumerate(cols):
    i = k // ncols
    j = k % ncols

    plot(ax[i][j], feat)

  for k in range(nrows * ncols - 1, len(cols) - 1, -1):
    i = k // ncols
    j = k % ncols
    plt.delaxes(ax[i][j])

  plt.tight_layout()

In [ ]:
# Perform adjustments to the histogram for "discrete" variables, that is,
# numerical variables consisting of a handful of discrete integers as values.
def discrete_plot(kwargs, ax, uniq_vals = 5):
  kwargs['bins'] = [i for i in range(0, uniq_vals + 2, 1)]
  ax.set_xticks([i for i in range(0, uniq_vals + 2, 2)])
  kwargs['stat'] = 'count'
  kwargs['kde'] = False

# Perform adjustments to the histogram for "year" variables like `YearBuilt`
def timeline_plot(kwargs, ax):
  kwargs['bins'] = np.arange(1870, 2030, 5) 

# Perform adjustments to the histogram for "continuous" numerical variables.
def continuous_plot(kwargs, ax, min, max, step, label_factor=2):
  kwargs['bins'] = np.arange(min, max, step)
  ax.set_xticks(np.arange(min, max + step, label_factor * step))

discrete_features = ['BedroomAbvGr', 'BsmtHalfBath', 'BsmtFullBath', 
                     'GarageCars', 'KitchenAbvGr', 'FullBath', 'HalfBath',
                     'TotRmsAbvGrd', 'KitchenAbvGr', 'Fireplaces',
                     'OverallQual', 'OverallCond', 'MoSold']

def pre_adjustment(ax, feat):
  kwargs = { 'stat' : 'density', 'kde' : True }

  if feat in discrete_features:
    discrete_plot(kwargs, ax, len(np.unique(X_train[feat])))

  # feature-specific adjustments to the graph to make it more readable.

  if feat == 'BsmtFinSF1':
    continuous_plot(kwargs, ax, 0, 2400, 100, 4)

  if feat == 'BsmtFinSF2':
    continuous_plot(kwargs, ax, 0, 1200, 100)

  if feat == 'YearRemodAdd':
    continuous_plot(kwargs, ax, 1950, 2030, 5)

  if feat == 'TotalBsmtSF':
    continuous_plot(kwargs, ax, 0, 3300, 100, 4)

  if feat == 'ScreenPorch':
    ax.set_ylim([0, 0.023])

  if feat == 'YrSold':
    continuous_plot(kwargs, ax, 2006, 2012, 1)
    kwargs['kde'] = False

  if feat == 'BsmtUnfSF':
    continuous_plot(kwargs, ax, 0, 2000, 100)

  if feat == 'LotFrontage':
    continuous_plot(kwargs, ax, 0, 320, 20)

  if feat == 'OpenPorchSF':
    continuous_plot(kwargs, ax, 0, 550, 50)

  if feat == 'PoolArea':
    continuous_plot(kwargs, ax, 0, 800, 100)
    kwargs['stat'] = 'count'
    kwargs['kde'] = False

  if feat == 'YearBuilt':
    timeline_plot(kwargs, ax)

  if feat == 'WoodDeckSF':
    continuous_plot(kwargs, ax, 0, 750, 50)

  if feat == 'LowQualFinSF':
    ax.set_ylim([0, 0.020])

  if feat == 'LotArea':
    continuous_plot(kwargs, ax, 1000, 216000, 5000, label_factor = 8)

  if feat == 'MiscVal':
    continuous_plot(kwargs, ax, 0, 17000, 1000)
    ax.set_ylim([0, 0.0010])

  if feat == 'GarageYrBlt':
    timeline_plot(kwargs, ax)

  if feat == '3SsnPorch':
    ax.set_ylim([0, 0.03])

  if feat == '1stFlrSF':
    continuous_plot(kwargs, ax, 0, 3600, 200)

  if feat == '2ndFlrSF':
    continuous_plot(kwargs, ax, 0, 3600, 200)

  if feat == 'GarageArea':
    continuous_plot(kwargs, ax, 0, 1400, 50, label_factor = 4)

  return kwargs

def plot_dist(ax, feat):
  kwargs = pre_adjustment(ax, feat)
  subax = sns.histplot(ax=ax, data=X_train, x=feat, **kwargs)
  return subax

plot_grid(plot_dist, list(set(numerical_cols) - set(wrong_num_cols)), nrows=6, ncols=6)

There are a few interesting distributions, that do not appear to be unimodal.
- `BsmtFinSF1` - Bimodal with peaks at `[0,100]` and `[600, 700]`
- `BsmtFinSF2` - Unimodal with peak at `[0, 100]`. This notable because the behaviour is different from `BsmtFinSF1`
- `YearRemodAdd` - This is trimodal. There are three peaks: `[1950, 1955]`, somewhere in `[1965, 1980]`, and `[2005, 2010]` 
- `BsmtUnfSF` - The distribution may be bimodal. The first peak is around `[0, 100]` and the second peak is around `[600, 700]`. Hence, houses appear to come in populations, one where there is unfinished basement surface area. One where there is unfinished surface area, where the median or mean is in `[600, 700]`. It is possible that the first peak is so large, because houses without a basement would have technically have 0 unfinished surface area.
- `WoodDeckSF` - Bimodal with two peaks: `[0, 50]` and `[100, 150]`
- `EnclosedPorch` - Bimodal with peaks: `[0, 50]` and `[100, 150]`
- `2ndFlrSF` - Bimodal for with two peaks `[0, 200]` and `[600, 800]`
- `TotalBsmtSF` - Bimodal. One population like has no basement, `[0, 100]` and one population has `[800, 900]` sq ft of basement.
- `GarageArea` - Possibly trimodal with two peaks `[0, 100]`, `[250, 300]`, and `[500, 550]`. 
- `MoSold` - The data appears to be bimodal. Housing sales peak at Month 6 (June) and Month 10 (October).

The year in which housing is being built show trimodal distributions. Moreover, they show similar distribution shapes.
- `YearBuilt` - This distribution appears to be trimodal. There were booms in housing construction centered around the years `[1920, 1925]`, `[1965, 1970]`, and `[2005, 2010]`. For the third boom, this might be potentially related to the 2008 U.S. housing bubble.
- `GarageYrBlt` - Has roughly the same distribution as `YearBuilt`. One explanation is that most houses are built with their garage. It is highly possible that these two features are multicollinear.

##### Barplots for Categorical Variables

In [ ]:
def plot_catcount(data=combined_train_with_absent):
  def _plot(ax, feat):
    subax = sns.countplot(ax=ax, data=data, x=feat)
    subax.set_xticklabels(subax.get_xticklabels(), rotation=45)
    return subax
  return _plot

plot_grid(plot_catcount(), object_cols + wrong_num_cols, nrows=7, ncols=7)

The barplots yields the following new information: 
- `MSZoning` - Values from most frequent to least frequent are: "Residential Low Density", "Residential Medium Density", "Floating Village Residential", "Commercial", and "Residential High Density". Interestingly, the commercial zoning has value "C (all)" instead of "C", the latter of which is used in the data documentation. It is unclear if "C (all)" has any special meaning over "C",
- `LotShape` - Most houses have regular lot shapes, about half as much have "Slightly irregular" shapes. A few houses have either moderately irregular or irregular. 
- `LotConfig` - ~750 houses have an "inside" lot config, a ~200 have "corner." The other values make up the rest.
- `BldgType` - Values from most frequent to least frequent are "Single-family detached", "Townhouse End Unit", "Duplex", "Townhouse Inside unit" (?), "Two-family conversion." The data documentation is incorrect, since `"Twnhs"` is the actual value representing "Townhouse Inside Unit". 
- `HouseStyle` - ~500 houses are 1story, ~350 are 2story. The other categories make up the rest. 
- `RooflStyle` - ~800 houses have gable roofs, ~200 have hip roofs.
- `Exterior1st` - From most to least frequent: "Vinyl Siding", "Hard Board", "Wood Siding", "Metal Siding", and then the rest. 
- `Exterior2nd` - From most to least frequent: "Vinyl Siding", "Wood Siding", "Wood Shingles", "Hard Board", "Plywood" and then the rest. 
- `MasVnrType` - ~600 houses have no masonry veneer, whereas ~300 have brickface, ~100 have stone. The rest have "Brick Common" 
- `ExterQual` - ~650 houses have average exterior quality, whereas ~350 have good exterior quality. The rest have either excellent or fair. 
- `Foundation` - ~450 houses have poured concrete foundations, ~450 have cinder block. 
- `BsmtQual` - ~450 houses have average basement quality, slighty less houses have good quality. The rest are fair or excellent.
- `BsmtFnType` - ~300 are unfinished, slightly less than that have "Good Living Quarters" 
- `HeatingQC` - ~500 are in excellent quality, ~300 average, ~200 good. 
- `KitchenQual` - ~500 are average, ~400 are good.
- `GarageType` - ~600 are attached, ~300 are detached. 
- `GarageFinish` - The finish is split between ~400 unfinished, ~300 rough finished, ~250 finished. 
- `Fence` - ~100 provide minimum privacy, ~45 have good privacy, ~40 have good wood, and then the rest.
- `MSSubClass` - Most of the housing sales occur for code 20 (1-story 1946 and newer) and 60 (2-story 1946 and newer). One way of explaining this is that newer houses are more desirable than older ones, particular those built earlier than 1946, which is more than 60 years ago from the earliest sell date in the data, 2006. 
- There are a few graphs with skewed classes, which might distort the linear regression model, such as: `Alley`, `PoolQc`, `MiscFeature`. The overwhelming majority of observations have the value `"Absent"` and our model might not be able to estimate the benefit when these assets are not absent, since there are too few data points to learn from. 
- The rest of the quality variables follow a similar pattern to ones already mentioned. There are no unusual bar graphs. 

- **[TASK]** - Create a boxplot for `SalePrice` against `Street`, `Neighborhood`, `Condition1`, `Exterior1st`, `Exterior2nd`
- **[TASK]** - Consider dropping `Alley`, `PoolQc`, `MiscFeature` 

### Bivariate Analysis

#### SalePrice vs. Numeric Predictors

In [ ]:
# Scatterplots with each numerical feature 
def plot_regression(ax, feat):
  subax = sns.regplot(ax=ax, data=combined_train, x=feat, y='SalePrice')
  return subax

plot_grid(plot_regression, list(set(numerical_cols) - set(wrong_num_cols)), nrows=6, ncols=6, figlen=10)

We make the following observations. 

First, the following features have scatterplats that indicate that homoscedasticity is violated, since the points are spread out so they have a "fanning" pattern

- `YearRemodAdd` 
- `GarageYrBlt`
- `1stFlrSF`
- `MasVnrArea`
- `GarageArea` 
- `LotFrontage`
- `YearBuilt`? 
- `OpenPorchSF`
- `EnclosedPorch`
- `TotalBsmtSF`
- `GrLivArea` 
- `WoodDeckSF`? 

The following features might satisfy homoscedasticity

- `BsmtUnfSF`
- `BsmtFinSF1`
- `BsmtFinSF2`?
- `2ndFlrSF`? 

All other features have an unclear with homoscedasticity. For example, `LowQualFinSF` might violate homoscedasticity, but this could be a visual distortion due to a small number of data points. In the "Model Assumptions" section, we make additional plots to confirm any violations of homoscedasticity.

Additional, we also note that the following features have a down-sloping regression line.

- `MiscVal` - This result might be distorted due to the fact there are few data points. 
- `BsmtUnfSF`
- `EnclosedPorch` - This result might be distorted due to the high number of data points without any enclosed porch.
- `KitchenAbvGr` - Might be distorted since the x-variable has discrete values.
- `BsmtHalfBath` - Might be distroted since the x-variable has discrete values.
- `YrSold`? - The slope appears to be close to 0, so the relationship is very mild.
- `OverallCond` - Might be distorted by the number of "average" condition scatterpoints.
- `LowQualFinSF` - Might be distorted by the small number of points.  

Hence, it seems as though many of the negative relationships are being distorted due to quirks with the data, e.g, discrete feature values or a high number of points with feature value 0. All other features appear to have positive relationships, as indicated by an upward-sloping regression line. 

The following features' scatterplots appear to be distorted by a high number of points with value 0. 

- `TotalBsmtSF`
- `WoodDeckSF`
- `BsmtFinSF1`
- `BsmtFinSF2`
- `LowQualFinSF`
- `3SsnPorch`
- `PoolArea`
- `2ndFlrSF`

We should check the regression after the points with value 0 are removed to better understand the relationship of the feature with `SalePrice`. This is justified because often when a data point has predictor value 0, it indicates that it doesn't exist, e.g, `WoodDeckSF = 0` indicates no wooden deck. These points will have high variability in `SalePrice`, because other features will take on wildly different values. Hence, these points are somewhat exceptional. 

We can therefore treat each listed feature above as having two populations: one with zero predictor value and with non-zero predictor value.

In [ ]:
# Adjusting some of the plots above, by removing data points where feature value is 0. 

def plot_scatter(ax, feat):
  nonzero = combined_train[combined_train[feat] > 0]
  subax = sns.regplot(ax=ax, data=nonzero, x=feat, y='SalePrice')
  return subax

with_zeros = ['TotalBsmtSF', 'WoodDeckSF', 'BsmtFinSF1', 'BsmtFinSF2', 'LowQualFinSF', '3SsnPorch', 'PoolArea', '2ndFlrSF']
plot_grid(plot_scatter, with_zeros, nrows=2, ncols=4, figlen=10)

#### SalePrice vs. Categorical Predictors

In [ ]:
# Boxplots of the categorical features with `SalePrice` on the y-axis.
def plot_boxplot(ax, feat):
  # Plotting the actual boxes in order of lowest to highest median
  order = combined_train_with_absent.groupby(by=[feat])["SalePrice"].median().sort_values().index
  subax = sns.boxplot(ax=ax, data=combined_train_with_absent, x=feat, y='SalePrice', order=order)

  # Rotating the x-labels to be readable
  subax.set_xticklabels(subax.get_xticklabels(), rotation=45)

  for pos, label in enumerate(ax.get_xticklabels()):  
      series = combined_train_with_absent[combined_train_with_absent[feat].astype('string') == label.get_text()]
      median = series['SalePrice'].median()
      mean = series['SalePrice'].mean()
      n = len(series)

      ax.text(pos,
              median + 0.04,
              "n: {n:d}\nmu: {mean:.0f}".format(n=n, mean=mean),
              horizontalalignment='center',
              size='x-small',
              color='w',
              weight='semibold')

  return subax

plot_grid(plot_boxplot, object_cols + wrong_num_cols, nrows=7, ncols=7, figlen=10)

Above, we have a boxplot of every categorical feature against the `SalePrice`. We have annotated each box with the number of elements with that categorical value and the mean `SalePrice` for that category. 

We make the following observations. 

- `MSZoning` - Surprisingly, "Floating Village" has a high average and median price. It is not clear what this residential zone is meant to represent. "Commercial" zones tend to have the lowest `SalePrice` . Whereas residential zones occupy the "middle" with low-density residential areas being the highest. Low-density residential areas being the highest might make sense, if houses in this area take up a lot of land. Then, an individual person must pay for more land, than those in low-density zones. 
- `Street` - Paved road access to property is associated with a higher median and mean `SalePrice`. This might illustrate a dichotomy between rural (gravel roads) vs. urban (paved roads) housing. 
- `LotShape` - The more irregular the lot is, the higher the median `SalePrice`. We could target encode `LotShape` by median `SalePrice`
- `LandContour` - Level and banked land is associated with lower `SalePrice`, whereas depressed or hillside land is associated with higher `SalePrice` 
- `Utilities` - This is dominated by housing that has all utilities, so we cannot contrast different groups.
- `LotConfig` - FR3 has too few examples to grasp its properties. Inside, Corner, and Frontage on 2 Sides are roughly similar in price. A CulDSac config is associated with a higher `SalePrice`. 
- `LandSlope` - A `Gtl` slope is priced less than moderate or severe, which are equally as expensive. Encoding by median `SalePrice` does not follow the natural ordering of `Gtl`, `Mod`, `Sev`. We might want to consider mean encoding or simply using median encoding. 
- `Neighborhood` - As expected, the boxplot visually confirms that the neighborhood can make a significant difference as to what the price is. Some neighborhoods tend to have more expensive houses than others.
- `Condition1` - Arterials have the lowest `SalePrice`. This does not disprove our initial assumptions, because an arterial road is likely to be noisy due to high traffic flow. The noise polluation would most likely drive down `SalePrice`. Similar logic applies to a feeder street. Both `RRAn` and `RRAe` imply that a house is relatively close to a railroad, which agrees with our assumption. On the other hand, parks and positive features, indicated by `PosN` and `PosA`, drives up `SalePrice`. The `RRNn` and `RRNe` variables are difficult to interpret, since it is unclear what "200'" means in the data description. This likely refers to properties that are within 200 units of a railroad, but not directly beside it. This might explain why the `SalePrice` is not lower than the `Norm` condition, but it does not explain why it is higher. 
- `BldgType` - The boxplot seems to reveal the following trend: houses that offer more privacy and has fewer families living in a single unit have a higher `SalePrice`. A "Single-family" detached has the highest sale price. A townhouse end-unit is a higher sale price than the inside unit, because it provides more privacy according to [this](https://firsthousecoach.com/is-an-end-unit-townhouse-worth-more/) article. The Duplex home does break this trend, as a duplex could be inhabited a single family. 
- `RoofMatl` - Wooden shake and wood shingle houses are rare and are associated with housing with very high `SalePrice`. A tar and gravel roof is also rare and it seems to be associated with slightly higher `SalePrice`. Roofs typically use composite shingles. 
- `MasVnrType` - Veneer made from stone or break-face appears to be associated with expensive housing.
- `ExterQual` - Higher quality is associated with higher `SalePrice`
- `ExterCond` - This feature appears to be less influential compared to `ExterQual` 
- `BsmtQual` - Higher quality, higher `SalePrice`
- `BsmtCond` - Clear relationship with `SalePrice`, but less influential than `BsmtQual`. 

- Generally speaking higher values for "quality" or "condition" features leads to an increase to `SalePrice` as expected. 

For most of the features, the categorical classes can be target encoding using the median `SalePrice`. This kind of encoding could be useful for capturing nonlinearity between a feature and the `SalePrice` 

- **[TASK]** - Determine what a floating village is and why it would be so expensive.
- **[TASK]** - Add categorical predictors one at a time to increase performance without introducing problems due to many one-hot encoded features. 

### Multivariate Analysis

#### Numeric vs. Numeric

In [ ]:
numerical_cols_2 = list(set(numerical_cols) - set(wrong_num_cols))
corrmat = combined_train[numerical_cols_2 + [target_col]].corr()
fig, ax = plt.subplots(figsize=(30, 30))
sns.heatmap(corrmat, ax=ax, square=True, annot=True, vmin=-1, vmax=1)
ax.set_title("Correlation matrix for all numeric features")

In [ ]:
# Define thresholds for pairs of features that have a notable amount of correlation
low_threshold = 0.30
med_threshold = 0.50
high_threshold = 0.70

def get_feats_with_min_cor(corrmat, threshold):
  _corrmat = (corrmat.abs() * 1000).round(0)
  _within_threshold = 1000 * threshold <= _corrmat
  _not_on_diagonal = _corrmat < 1000
  _not_null = corrmat != np.nan
  mask = (_within_threshold & _not_on_diagonal & _not_null).sum() > 0

  feats = mask[mask].index
  return list(feats)

def get_feat_corr_hierarchy(corrmat):
  # Determine the features with at least one entry that has a correlation above the threshold. Such features are worth further investigation.
  all_feats = get_feats_with_min_cor(corrmat, 0)
  low_feats = get_feats_with_min_cor(corrmat, low_threshold)    # features which has at least 1 entry, with at least 0.30 correlation
  med_feats = get_feats_with_min_cor(corrmat, med_threshold)    # features which has at least 1 entry, with at least 0.50 correlation
  high_feats = get_feats_with_min_cor(corrmat, high_threshold)  # features which has at least 1 entry, with at least 0.70 correlation
  none_feats = list(set(all_feats) - set(low_feats))            # features with no entry having correlation greater than or equal to 0.30
  return low_feats, med_feats, high_feats, none_feats

In [ ]:
low_feats, med_feats, high_feats, none_feats = get_feat_corr_hierarchy(corrmat)

print(none_feats)
print(low_feats)
print(med_feats)
print(high_feats)

In [ ]:
low_corrmat = combined_train[low_feats].corr()
med_corrmat = combined_train[med_feats].corr()
high_corrmat = combined_train[high_feats].corr()

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
sns.heatmap(low_corrmat, ax=ax, square=True, annot=True, vmin=-1, vmax=1)
ax.set_title("Features with at least 1 low correlation")

Above is the low correlation matrix. We will ignore this matrix for now, because pairs of features with correlation between $[0.10, 0.30]$ might be negligible for our linear regression model. Features with low correlation in the range $[0.30, 0.50]$ might negatively impact the linear regression model, but perhaps far less than moderately or highly correlated features.

If additional performance increases are necessary, we may consider dropping features from the low correlation matrix. 

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
sns.heatmap(med_corrmat, ax=ax, square=True, annot=True, vmin=-1, vmax=1)
ax.set_title("Features with at least 1 medium correlation")

In the medium correlation matrix, we note the following medium correlation relationships, that is, entries with correlation values in $[0.50, 0.70)$.

- `YearRemodAdd`, `GarageYrBlt` - This relationship is peculiar, because there is a moderate, positive linear relationship. One hypothesis that explains this correlation is that garages are often part of a remodelling, so `YearRemodAdd` is approximately equal to `GarageYrBlt` for several entries, which leads to this high correlation. 
    
    From the high correlation analysis, we are already discarding `GarageYrBlt`, so there is no action to be done.

- `YearRemodAdd`, `YearBuilt` - As shown in the high correlation matrix analysis, `YearBuilt` and `GarageYrBlt` are highly correlated, and we saw earlier that `GarageYrBlt` and `YearRemodAdd` are correlated, so this correlation might be explained by transitivity. 

    Intuitively, the `YearRemodAdd` feature does not appear to be redundant with `YearBuilt`, so we will not discard anything.

- `GarageYrBlt`, `GarageArea` - This relationship is **surprising**. It is difficult to explain why there appears to be a moderate correlation. One hypothesis is that as a city develops over time, there is a greater need to drive, such as going to work. Therefore, as time goes on, more houses are built or remodelled to accomodate more cars. 

    From the high correlation analysis, we already decided to discard `GarageYrBlt`, so no action is required.

- `GarageYrBuilt`, `GarageCars` - `GarageCars` is strongly correlated with `GarageArea` so the same reasoning as above applies. 

    From the high correlation analysis, we already decided to discard both features, so no action is required.

- `GarageYrBuilt`, `OverallQual` - We know from the high correlation analysis that `GarageYrBuilt` is highly correlated with `YrBuilt`. We also see that `YrBuilt` has a moderate correlation with `OverallQual`. Therefore, by transitivity, `GarageYrBuilt`, `OverallQual` are moderately correlated. To explain the correlation between `YrBuilt` and `Overall`, one hypothesis is that as a city develops over time, the housing constructed has better quality overall. 

    From the high correlation analysis, we already decided to discard `GarageYrBlt`, so no action is required.

- `BsmtFullBath`, `BsmtFinSF1` 

    We could discard `BsmtFullBath` since it is a discrete variable, as long as it provides a real performance improvement

- `TotRmsAbvGrd`, X 
  
    Skipping this row with `TotRmsAbvGrd`, since we have already decided to discard this feature in the high correlation analysis.

- `BsmtUnfSF`, `BsmtFinSF1`- This is a negative relationship, which makes intuitive sense since the features refer to opposite quantities.

    We could try discarding one feature or the other.

- `1stFlrSF`, `GrLivArea` 

    We could try discarding one feature or the other.

- `GarageArea`, `OverallQual`

    Intuitively, these features seem to be non-redundant so we will not discard either of them. 

- `YearBuilt`, `OverallQual`

    Intuitively, these features seem to be non-redundant, so we will not discard either of them.

- `YearBuilt`, `GarageCars`

    `GarageCars` is already discarded based on the high-level analysis, so not action required.

- `HalfBath`, `2ndFlrSF` 

    The correlation here is somewhat high, $>=0.60$. We can try discarding `HalfBath` or discarding `2ndFlrSF` 

- `BedroomAbvGr`, `GrLivArea`

    We could try discarding `BedroomAbvGr`, since `GrLivArea` has a stronger correlation with `SalePrice` and `GrLivArea` is a continuous variable.

- `TotalBsmtSF`, `OverallQual`

    We have already discarded `TotalBsmtSF`, so no action is required.

- `GrLivArea`, X - The `GrLivArea` is highly correlated with `FullBath`, `GarageCars`, and `OverallQual`. 

    Perhaps consider dropping other features besides `GrLivArea` 

- `FullBath`, `OverallQual` 

    Perhaps consider dropping `FullBath`, since `FullBath` has a weaker correlation with `SalePrice` and `FullBath` is correlated with other features such as `GrLivArea` already. 

- `GarageCars`, X

    We are alreay dropping `GarageCars`, so no action is required here.


In summary, there appears to be an interconnected relationship between `YearBuilt`, `OverallQual`, and `GarageArea` 


- **[ENG]** - Try discarding `BsmtFullBath`
- **[ENG]** - Try discarding `TotRmsAbvGrd` and keeping the individual above-grade room counts. Compare this to just keeping `TotRmsAbvGrd` to determine if there is any performance boost. 
- **[ENG]** - Try discarding `GrLivArea` or `1stFlrSF` or keeping both to see if there is any performance boost.
- **[ENG]** - Try discarding `HalfBath`. If we are preprocessing in an alternate way, where we discard `2ndFlrSF`, then try to keep `HalfBath` 
- **[ENG]** - Try discarding either `BsmtUnfSF` or `BsmtFinSF1` or keeping both.
- **[ENG]** - Try discarding `BedroomAbvGr` over `GrLivArea`
- **[ENG]** - Consider dropping `FullBath` and `OverallQual`, which are correlated with `GrLivArea`. Note that `FullBath` is correlated with `OverallQual`



In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
sns.heatmap(high_corrmat, ax=ax, square=True, annot=True, vmin=-1, vmax=1)
ax.set_title("Features with at least 1 high correlation")

From the high correlation matrix, we see the following high correlation relationships between predictors:
- `GarageYrBlt`, `YearBuilt` - Earlier, in the bar chart visualization, we saw that these two features have the same distribution and speculated that most houses are built alongside their garage. We see that the correlation value does not disprove this hypothesis. We can remove `GarageYrBlt` to reduce multicollinearity.
- `GrLivArea`, `TotRmsAbvGrd` - According to the data documentation, `GrLivArea` represents "Above grade (ground) living area square feet" and `TotRmsAbvGrd` represents "Total rooms above grade (does not include bathrooms)". Thus, the two features are almost equivalent, which explains their high correlation. We discard `TotRmsAbvGrd`, because a continuous variable may be more suitable for linear regression. 
- `1stFlrSF`, `TotalBsmtSF` - This correlation is rather **surprising**. It appears that the surface area of a basement is correlated with the surface area of the first floor. Yet, `1stFlr` and `2ndFlrSF` has a correlation value of only -0.21. We can discard `TotalBsmtSF` to reduce multicollinearity for linear regression.
- `GarageArea`, `GarageCars` - This correlation is to be expected. A garage that accomodates cars obviously requires more surface area. Interestingly, the correlation is very high, 0.9. This could be explained by some kind of construction standard, e.g., for every car we want to accomodate, add 900 sq ft to the garage. We discard `GarageCars` to reduce multicollinearity and since continuous variables are easier to work with. 
- `2ndFlrSF`, `GrLivArea` - Interestingly, `GrLivArea` has a stronger correlation with `2ndFlrSF` compared to `1stFlrSF`. We could discard `2ndFlrSF`. We do not discard `GrLivArea`, because it has a higher correlation with the target, `SalePrice`

- **[ENG]** - Discard `GarageYrBlt`
- **[ENG]** - Discard `TotRmsAbvGrd`
- **[ENG]** - Discard `TotalBsmtSF`
- **[ENG]** - Discard `GarageCars`
- **[ENG]** - Discard `2ndFlrSF`
- **[TASK]** - For each pair above, plot a scatterplot to illustrate their relationship, because they might not be in a linear relationship. 

In [ ]:
predictors_ranking = corrmat['SalePrice'].sort_values(ascending=False)
predictors_ranking

As shown above, `OverallQual`, `GrLivArea` have a high correlation

- **[TASK]** - Check the relationship between `SalePrice` and each of the features above.
- **[ENG]** - Include `GrLivArea`, `OverallQual`, `GarageArea`, `1stFlrSF`, 
`YearBuilt` all as predictors.
- **[ENG]** - Run experiments using `FullBath`, `YearRemodAdd` as features
- **[ENG]** - Run experiments using `MasVnrArea`, `Fireplaces`, `BsmtFinSF1`, `LotFrontage`, `2ndFlrSF`, `OpenPorchSF`, `WoodDeckSF` as features.
- **[ENG]** - Run experiments using features with absolute correlation $<=0.30$ as predictors.

In [ ]:
# Scatterplots between predictor values
def plot_scatter(ax, feat):
  subax = sns.scatterplot(ax=ax, data=combined_train, x=feat[0], y=feat[1])
  return subax

high_corr_pairs = [['1stFlrSF', 'TotalBsmtSF'], ['GrLivArea', 'TotRmsAbvGrd'], 
                   ['GarageYrBlt', 'YearBuilt'], ['GarageCars', 'GarageArea'],
                   ['2ndFlrSF', 'GrLivArea']]
med_corr_pairs = [['YearRemodAdd', 'YearBuilt'], ['BsmtFullBath', 'BsmtFinSF1'],
                  ['BsmtUnfSF', 'BsmtFinSF1'], ['1stFlrSF', 'GrLivArea'],
                  ['GarageArea', 'OverallQual'], ['YearBuilt', 'OverallQual'],
                  ['HalfBath', '2ndFlrSF'], ['BedroomAbvGr', 'GrLivArea'],
                  ['FullBath', 'OverallQual'], ['GrLivArea', 'FullBath'], 
                  ['GrLivArea', 'GarageCars'], ['GrLivArea', 'OverallQual']]

pairs = high_corr_pairs + med_corr_pairs

plot_grid(plot_scatter, pairs, nrows=5, ncols=4, figlen=10)

In the scatterplots above, we have chosen to plot highly- and moderately-correlated features, as plotting all 36 x 36 scatterplots using `sns.pairplot` has a long runtime.

We make the following observations:
- `1stFlrSF`, `TotalBsmtSF` - There are appear to be three populations in this scatterplot: houses with no basement, houses where `1stFlrSF == TotalBsmtSF`, and all other houses. There are a few outliers: one where the basement is far larger than the first floor, and one where the first floor is far larger than the basement. 
- `GrLivArea`, `TotRmsAbvGrd` - There is an obivous, positive linear trend. There appear to be two outliers at the far right that do not obey the trend. 
- `GarageYrBuilt`, `YearBuilt` - Illustrates two populations: one where the garage was built alongside the house, one where the garage was built after the house. There is also a third population of houses that do not have garages. 
- `GarageCars`, `GarageArea` - Illustrates an obvious, positive linear trend.
- `2ndFlrSF`, `GrLivArea` - There are two populations: (1) houses without a second floor, (2) houses with a second floor. In the latter case, there is a clear linear trend.
- `YearRemodAdd`, `YearBuilt` - This illustrates a fact that was listed in the data documentation: for `YearRemodAdd`, it is "same as construction date [`YearBuilt`] if no remodeling or additions". There are two populations: (1) houses that were never remodelled, (2) houses that were remodelled. The later population shows no trend, which is to be expected. 
- `BsmtUnfSF`, `BsmtFinSF1` - This scatterplot suggests that there is no real relationship between the two features. The moderate correlation level might be a fluke. Hence, there is no need to drop either feature, since they have little redundancy. 
- `1stFlrSf`, `GrLivArea` - This scatterplot reveals a fact that should have been obvious, in hindsight. If a house has a single floor above ground, then `1stFlrSF` is exactly equal to `GrLivArea` 
- `GarageArea`, `OverallQual` - This has a clear, positive linear trend. One hypothesis to explain this trend, is that the two features are linked to a third variable, the general "value" of a property, that is, rich estates or mansions have large garages and high quality, whereas cheaper housing tends to have less garages and are lower quality. 
- `YearBuilt`, `OverallQual` - This relationship makes intuitive sense. New housing has yet to be damaged by age.
- The rest of the graphs show obvious relationships like `YearBuilt`, `OverallQual`

- **[TASK]** - For pairs of features that show an obvious line of data points, we can construct new features by taking the difference between the pair. 
- **[TASK]** - Try to manipulate `1stFlrSF`, `2ndFlrSF`, and `GrLivArea` to find their relationship. Perhaps consider a 3D plot. 

#### Categorical vs. Categorical

In [ ]:
from scipy.stats import chi2_contingency

def make_contigency_table(data, x, y):
  df = []
  uniq_xs = np.unique(data[x].dropna(axis=0))
  uniq_ys = np.unique(data[y].dropna(axis=0))

  for target in uniq_ys:
    row = []
    for pred in uniq_xs:
      X_with_pred = data[data[x] == pred]                     # get all records in `data` with the specific value for the predictor
      X_with_target = X_with_pred[X_with_pred[y] == target]   # get all records in `X_with_pred` with the specified predictor AND target
      cell = len(X_with_target)                               # now count the number of records with `x=pred`, `y=target`
      row.append(cell)
    df.append(row)

  return pd.DataFrame(df).transpose()

def cramers_v(cont_table):
  chi2 = chi2_contingency(cont_table, correction=False)[0]
  N = cont_table.to_numpy().sum()
  k = np.min(cont_table.shape) - 1
  V = np.sqrt((chi2 / N) / k)
  return V

def cat_corrmat(data, object_cols):
  matrix = []
  for row_feat in object_cols:
    row = []
    for col_feat in object_cols:
      try:
        tbl = make_contigency_table(data, row_feat, col_feat)
        v = cramers_v(tbl)
        row.append(v)
      except ValueError:
        row.append(np.nan) # fill the cell with some error value 
    matrix.append(row)
  return pd.DataFrame(matrix, index=object_cols, columns=object_cols)

def plot_cat_corrmatrix(corrmat, object_cols):
  fig, ax = plt.subplots(figsize=(30, 30))
  subax = sns.heatmap(corrmat, ax=ax, annot=True)

catmatrix = cat_corrmat(combined_train_with_absent, object_cols)
plot_cat_corrmatrix(catmatrix, object_cols)

In [ ]:
low_cat_feats, med_cat_feats, high_cat_feats, _ = get_feat_corr_hierarchy(catmatrix)

print(low_cat_feats)
print(med_cat_feats)
print(high_cat_feats)

In [ ]:
def get_submatrix(corrmat, to_keep):
  rows_to_keep = corrmat.index.isin(to_keep)
  return corrmat[to_keep][rows_to_keep]

low_cat_corrmat = get_submatrix(catmatrix, low_cat_feats)
med_cat_corrmat = get_submatrix(catmatrix, med_cat_feats)
high_cat_corrmat = get_submatrix(catmatrix, high_cat_feats)

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
sns.heatmap(high_cat_corrmat, ax=ax, square=True, annot=True, vmin=0, vmax=1)
ax.set_title("Features with at least 1 high correlation")

The follow features are highly correlated and are not independent.

1. `Exterior1st`, `Exterior2nd`
2. `GarageQual`, `GarageCond`

- **[ENG]** - Discard `Exterior2nd`
- **[ENG]** - Discard `GarageQual` or `GarageCond`. Try to measure the correlation between `SalePrice` and the two features, then discard the one with the lowest correlation.


In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
sns.heatmap(med_cat_corrmat, ax=ax, square=True, annot=True, vmin=0, vmax=1)
ax.set_title("Features with at least 1 medium correlation")

The following features have medium correlation

- `Neighborhood`, `MSZoning` - These two features are somewhat redundant, since zoning might not vary much within the same neighborhood. 
- `KitchenQual`, `ExterQual` - These variables appear to be non-redundant. One is the quality of the kitchen, one is the quality of the exterior. No action here.

There are three "clusters" of highly related features.

The first cluster contains the categorical features for the basement, which all have medium correlations. This subset of features might contain redundant information between them.

- `BsmtQual`, `Foundation` - These features do not appear to be redundant information, so we might keep both of them. 
- `BsmtCond`, `BsmtQual` - We should consider discarding one of these.
- `BsmtExposure`, `BsmtQual` - Perhaps consider dropping `BsmtExposure` since `BsmtQual` can be interpreted as a continuous feature. 
- `BsmtExposure`, `BsmtCond` - Perhaps consider dropping `BsmtExposure` since `BsmtCond` can be interpreted as a continuous feature.
- `BsmtFinType1`, `BsmtQual` - Perhaps consider dropping `BsmtFinType`
- `BsmtFinType1`, `BsmtCond` - Perhaps consider dropping `BsmtFinType`
- `BsmtFinType1`, `BsmtExposure` - Perhaps consider dropping `BsmtFinType`
- `BsmtFinType2`, X - Same as `BsmtFinType`

Secondly, the categorical features for the garage all have medium correlations.

- `GarageFinish`, `GarageType` - Perhaps discard `GarageType` in favour of `GarageFinish`, because finish can be interpreted as ordinal, whereas `GarageType` cannot.
- `GarageQual`, `GarageFinish` - Perhaps discard `GarageFinish`? 
- `GarageCond`, `GarageFinish`

Lastly, the there appears to be another cluster involving the following features, where any pairs of 2 features have moderate or to close to moderate values for Cramer's V

- `Exterior1st`
- `Exterior2nd`
- `ExteriorQual`
- `Foundation`
- `BsmtQual`

Interestingly, `Neighborhood` has low to moderate correlations with several features. This is in accordance with conventional wisdom which implies that neighborhood is a critical factor in finding good housing, as it can determine overall house quality and price. 

- **[ENG]** - Empirically check if discarding `Neighborhood` or `MSZoning` improves performance
- **[ENG]** - Discard `BsmtCond` or `BsmtQual`, depending on which has less correlation with `SalePrice`
- **[ENG]** - Try discarding `BsmtExposure`, `BsmtFinType1`, and `BsmtFinType2`, as they are nominal variables, whereas `BsmtQual` and `BsmtCond` are continuous variables that capture some of the information. 
- **[ENG]** - Empirically check if discarding `GarageFinish` and/or `GarageType` in favour of `GarageQual` leads to performance boosts.
- **[ENG]** - Try manipulating the basement and garage categorical variables to remove redundancy and improve performance. Create a preprocessor that empirically checks every non-empty subset of the collinear features, then tests to see which one yields the best performance. 
- **[ENG]** - For the first model, try building a simple model using only numerical and ordinal features. Then, add nominal features as required to boost performance. 


In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
sns.heatmap(low_cat_corrmat, ax=ax, square=True, annot=True, vmin=0, vmax=1)
ax.set_title("Features with at least 1 low correlation")

We skip the analysis of the low correlation features for now, until we need additional performance boosts. 

#### Numeric vs. Categorical

To exhaustively assess numerical vs. categorical relationships, we would need to plot distributions for every pair of 1 numerical and 1 categorical feature. However, there are at least 30 numerical and 30 categorical features, necessitating at least 900 plots. This would take too long to run.

Instead, we first compute the matrix of Kruskall-Wallis H test values. Then, for any entries that are particularly high, we plot their distributions to confirm that the continuous and numeric feature pair meets the H test's assumptions.

The advantage of this approach is that the notebook will take less time to run.

The disadvantage of this approach is that we may miss false negatives, that is, feature pairs where the H test assumption is violated and the two features in fact have a high association, but the corresponding H-test value is not high enough, so we wouldn't think to check this pair.



In [ ]:
from scipy.stats import kruskal

def partition_by_cat(data, cat_feat, cont_feat):
  series = data[cat_feat]
  group_labels = np.unique(series.dropna())
  groups = []
  for label in group_labels:
    group = data[series == label][cont_feat]
    groups.append(group)
  return groups

def kruskal_wallis_h(data, cat_feat, cont_feat):
  groups = partition_by_cat(data, cat_feat, cont_feat)
  k, p = kruskal(*groups, nan_policy='omit')
  return k, p

def kruskal_matrix(data, object_feats, numerical_feats, option='statistic',
                   transpose = False):
  rows = []
  for ofeat in object_feats:
    row = []
    for nfeat in numerical_feats:
      h, p = kruskal_wallis_h(data, ofeat, nfeat)

      cell_val = None
      if option == 'statistic':
        cell_val = h
      elif option == 'pvalue':
        cell_val = p
      else:
        raise ValueError('incorrect value for option in `kruskal_matrix`')

      row.append(cell_val)

    rows.append(row)

  retVal = pd.DataFrame(rows, index=object_feats, columns=numerical_feats)

  return retVal if not transpose else retVal.transpose()

In [ ]:


obj_cols = object_cols + wrong_num_cols
num_cols = list(set(numerical_cols) - set(wrong_num_cols)) + [target_col]
kmatrix = kruskal_matrix(combined_train_with_absent, 
                         obj_cols,
                         num_cols,
                         option='pvalue',
                         transpose=True)

fig, ax = plt.subplots(figsize=(30, 30))
sns.heatmap(- np.log(kmatrix), ax=ax, annot=True, vmin=0, vmax=2)
ax.set_title("Heatmap of negative log p-values from Kruskall-Wallis H test")

In the heatmap above, any value of 2 or higher corresponds to a $p$-value of 0.01 or lower. We take this to mean there is strong evidence that the alternate hypothesis, that the numerical and continuous variables are dependent is true. 

Unfortunately, there are more statistically significant associations in the heatmap than originally anticipated. We will concentrate on the row with `SalePrice` and use the other rows as needed. According to the heatmap, there is strong statistical evidence of a relationship between almost every categorical feature and `SalePrice`.

We now verify the assumptions for the Kruskall-Wallis H Test to check if the test gives valid results. To do this, given a categorical feature, we must check if the probability distribution of `SalePrice` is the same shape over all the values of the categorical feature.

In [ ]:
def plot_multidist(ax, feat): 
  return sns.histplot(combined_train_with_absent, ax=ax, x=target_col, hue=feat,
                      fill=True)
  
plot_grid(plot_multidist, obj_cols, ncols=7, nrows=7)

Unfortunately, it appears that many plots show that the distributions do not have the same shape. For many plots, this is due to distortions caused by small group sizes making it difficult to obtain a reliable comparison. 

For a future EDA, we may consider randomly sampling each group so that we compare the probability distributions of similarly sized data-sets. We can use the resulting plot as an estimate of the "true" behaviour among groups in the overall population.

### Tasks

#### Determine What a "High-end" Home Typically Looks Like

- Refer back to the univariate analysis for "luxury" assets
- Check if these features really correspond to "luxury" assets by checking the `SalePrice` of homes that have these assets (that is homes with boolean = 1 or continuous_var > 0) OR if these luxury assets are really undesirable or obsolete characteristics. 

- Try to identifying the outliers using the `outlier_threshold` variable then check feature for any patterns. If you see a pattern, check against non-outlier houses to see if that pattern is not present in the lower end houses. 

In [ ]:
series = combined_train_with_absent['SalePrice']
outlier_homes = combined_train_with_absent[series >= outlier_threshold]
outlier_homes

In [ ]:
outlier_homes[num_cols].describe() - combined_train_with_absent[num_cols].describe()

The outlier homes have:
- `GrLivArea` - A greater amount of space on average.
- `LotArea` - A significantly higher lot area on average.
- `OverallQual` - A higher overall quality on average
- `BsmtFinSF1` - High basement finished area 
- `GarageArea` - Higher garage area
- `MasVnrArea` - Higher masonry veneer area
- `YearBuilt` - The average year it was built is 25 years later. 

In general, the houses are newer and have higher quality overall. 

In [ ]:
plot_grid(plot_catcount(outlier_homes), obj_cols, nrows=7, ncols=7)

We make the following observations about the outlier homes:
- `MSZoning` - They must be either residential low density or resdiential medium density.
- `Street` - Must be paved
- `Alley` - Must be absent
- `LotShape` - There are more houses with slightly irregular shapes, than normal shapes. The proportion of moderately and highly irregular shapes is also higher than the overall dataset.
- `AllPub` - These houses all have their utilities.
- `Neighborhood` - The housing belong to `Crawfor`, `NridgHt`, `NoRidge`, `Timber`, `StoneBr`, `Somerst`, `OldTown`, `CollgCr`, `Gilbert`. The majority belong in `NridgHt`
- `MasVnrType` - The most to least frequent masonry veneer type is `Stone`, `BrkFace`, and `None`. This is the reverse order of `MasVnrType` in the overall dataset. 
- `ExterQual` - A high proportion of houses have good or better exterior quality. None have lower than average.
- `ExterCond` - Roughly the same proportions as the overall dataset, except condition is at least good.
- `Foundation` - Foundation is dominated by concrete, in contrast to the overall dataset, which is either concrete or cinderblock. 
- `BsmtQual` - Basement quality is at least average. Most of the basements have excellent quality.
- `BsmtBond` - Basement condition is at least average.
- `BsmtExposure` - Basement exposure is good on average. The `Gd` category makes a significantly higher proportion of the outliers, than in the overall dataset.
- `BsmtFinType` - A large proportion of basements are good living quarters. 
- `HeatingQC` - Most are excellent quality. All are at least average.
- `CentralAir` - All have a centralized system
- `Electrical` - All have `SBrkr` 
- `KitchenQual` - Many have excellent quality
- `HeatingQC` - Most have excellent quality
- `FireplaceQu` - All have the houses have fireplaces. They are all at least average quality. A large proportion are in excellent quality.
- `GarageType` - All of the houses have garages. Most of them are attached. 
- `GarageFinish` - Most of the houses have finished garages. 
- `GarageQual` - At least average
- `PavedDrive` - All of the houses have paved driveways. 
- `PoolQC`, `Fence`, `MiscFeature` - These are all absenta nd clearly do not contribute to the outlier's high price. 
- `SaleType` - An unusually high proportion of sales are "New" (home just constructed and sold) and "ConLI" (contrast low interest), and the rest are "WD". 
- `SaleCondition` - An unusually high proportion of sales are "Partial" (meaning the home was not completed when last assessed), the rest are "Normal" or "Alloca"
- `MSSubClass` - The class is predominantly 20 ("1-STORY 1946 & NEWER ALL STYLES") or 60 ("2-story 1946 & NEWER"). T

To summarize the outlier houses, they generally have at least average quality in nearly every aspect. They tend to have a basement that serves as good living quarters. They belong to a specific set of neighborhoods. They often have "conventional" design that is the most frequent value in the overall dataset, e.g, most housing have centralized air conditioning, so the outliers all have centralized air condition.

### Model Assumptions


#### Linear Regression

For simplicity, we will perform the checks of linear regression model assumptions against the numeric predictors. 

In [ ]:
def make_assume_pipeline():
    return Pipeline(steps=[('preprocessor', numerical_transformer),
                           ('model', LinearRegression())])

##### Linear Function

In [ ]:
def plot_for_linearity(X_train, X_valid, y_train, y_valid, numerical_cols):
  pipe_assume = make_assume_pipeline()
  pipe_assume.fit(X_train[numerical_cols], y_train)

  y_preds = pipe_assume.predict(X_valid[numerical_cols])

  residuals = y_valid - y_preds

  subax = sns.regplot(x=y_preds, y=residuals)
  subax.set_xlabel('Fitted Response')
  subax.set_ylabel('Residuals')
  subax.axhline(0, color='r')

  return residuals

plot_for_linearity(X_train, X_valid, y_train, y_valid, numerical_cols)

Linearity is clearly being violated. The data points appear to "dipping" down to a minimum around 200,000, then curving upwards. This appears to be a quadratic relationship between the fitted response and residuals.

We will attempt to transform the features to ensure linearity between `SalePrice` and each feature. Ideally, this will correct the residuals vs. fit plot to follow linearity. 

In fact, using a simple log transformation (rather than a square-root), we are able to satisfy linearity. 

In [ ]:
plot_for_linearity(X_train, X_valid, np.log(y_train), np.log(y_valid), numerical_cols)

There appears to be two outliers with a very low residual value. If we could ignore these outliers, then the plot will satisfy the linearity assumption better.

- **[ENG]** - Fit to the logarithm of `SalePrice` to satisfy linearity.



In [ ]:
residuals = plot_for_linearity(X_train, X_valid, np.log(y_train), np.log(y_valid), numerical_cols)
plt.ylim([-1, 0.5])

In [ ]:
# Determine the ID of the residual points
residuals[residuals <= -0.6]

In [ ]:
_y_valid = y_valid.copy()[(y_valid.index != 633) & (y_valid.index != 1299)]
_X_valid = X_valid.copy()[(X_valid.index != 633) & (X_valid.index != 1299)]
residuals = plot_for_linearity(X_train, _X_valid, 
                               np.log(y_train), np.log(_y_valid), 
                               numerical_cols)

As can be seen above, the plot appears to have a roughly random spread of data points around $y=0$, suggesting linearity.

However, we needed to discard outliers to achieve this, this could introduce systemic flaws to the model. 

##### Homoscedasticity
In the check for "Linear Function", we see that the residuals roughly lie around a horizontal band around $y=0$, roughly in range $[-0.4,0.4]$. This indicates that so long as we take the logarithm of `SalePrice`, the assumption of homoscedasticity is satisfied.

- **[ENG]** - Take the logarithm of `SalePrice` to satisfy homoscedasticity. 

##### No Autocorrelation
To evaluate auto-correlation, we can check if the residuals are auto-correlated when ordered by `YearBuilt`, `YearModAdd`, or `YrSold` and `MoSold`. This is not a perfect analysis, but we can eliminate some obvious possibilities for auto-correlation.

We ignore `GarageYrBlt` since we know that this is highly correlated with `YearBuilt` 

In [ ]:
X_train.columns

In [ ]:
def compute_residuals(X_train, X_valid, y_train, y_valid, cols):
  pipe = make_assume_pipeline()

  pipe.fit(X_train[cols], y_train)
  y_preds = pipe.predict(X_valid[cols])

  residuals_train = X_valid[cols].copy()
  residuals_train['Residuals'] = y_preds - y_valid

  return residuals_train

def compute_ordered_residuals(X_train, X_valid, y_train, y_valid, by, cols):
  residuals_train = compute_residuals(X_train, X_valid, y_train, y_valid,
                                      cols)

  return residuals_train[[by, 'Residuals']].sort_values(by=[by])

_num_cols = list(set(numerical_cols) - set(wrong_num_cols))

train_by_built  = compute_ordered_residuals(X_train, X_valid, 
                                            np.log(y_train), np.log(y_valid), 
                                            'YearBuilt', _num_cols)
train_by_mod    = compute_ordered_residuals(X_train, X_valid, 
                                            np.log(y_train), np.log(y_valid), 
                                            'YearRemodAdd', _num_cols)

X_train_time = X_train.copy()
X_train_time['TimeSold'] = X_train_time['YrSold'] + X_train_time['MoSold'] / 12
X_valid_time = X_valid.copy()
X_valid_time['TimeSold'] = X_valid_time['YrSold'] + X_valid_time['MoSold'] / 12

train_by_sold   = compute_ordered_residuals(X_train_time, X_valid_time,
                                            np.log(y_train), np.log(y_valid),
                                            'TimeSold', _num_cols + ['TimeSold'])

In [ ]:
from statsmodels.stats.stattools import durbin_watson

dw_built  = durbin_watson(train_by_built['Residuals'])
dw_mod    = durbin_watson(train_by_mod['Residuals'])
dw_sold   = durbin_watson(train_by_sold['Residuals'])

print("Durbin-Watson for sorted by by YearBuilt: %f" % dw_built)
print("Durbin-Watson for sorted by by YearRemodAdd: %f" % dw_mod)
print("Durbin-Watson for sorted by by TimeSold: %f" % dw_sold)

Since the test value is very close to 2, this demonstrates that there is no auto-correlation in the residuals, when ordered by the year built, year remodifications were add, or the year the house was sold.

Hence, this assumption appears to be satisfied. 

##### Normally-Distributed Residuals

In [ ]:
import statsmodels.api as sm

resid_train = compute_residuals(X_train, X_valid, 
                                np.log(y_train), np.log(y_valid),
                                _num_cols)
sm.qqplot(resid_train['Residuals'])
plt.show()

In [ ]:
sm.qqplot(resid_train['Residuals'])
plt.ylim([-0.5, 0.5])
plt.show()

The QQ-plot shows a roughly straight line, however, there are are two outliers with an extremely high residual. 

We will assume that normality is satisfied and investigate the outlier points (which is in the validation set) after the machine learning models are developed.

- **[TASK]** - Investigate outlier points in qqplot above. 

##### No Multicollinearity

As shown in the multivariate analysis, "Numeric vs. Numeric", there are a number of multicollinear features such as `YearBuilt` and `GarageYrBuilt`. 

Hence, the original raw dataset violates the multicollinearity assumptions. We can resolve this issue by discarding features with high multicollinearity. For more information on which features are discarded, see the "Numeric vs. Numeric" analysis. 

##### No Outliers
In the univariate analysis, there were a number of points that were greater than `threshold = q75 + 1.5 * IQR`. This indicates the presence of possible outliers, and so, this assumption is likely violated. 

Hence, either we can discard the outliers, which might introduce some systemic error if there is a consistent reason for the outliers' existence, or use a different model that is less sensitive to outliers. 

- **[ENG]** - Try fitting the linear regression model with and without the outlier points, to see if that improves performance.

#### Random Forest, Gradient Boosting, ANNs

There are no assumptions to check here, besides the standard assumptions: (1) sampling is representative, (2) values are independent and identically distributed. 

## Preprocessing Features

Based on the results of the EDA, we must perform the following steps for all models

- **[ENG]** - Construct the initial model using `GrLivArea`, `OverallQual`,`GarageArea`, `1stFlrSF`, `YearBuilt` all as predictors.
- **[ENG]** - Impute `LotFrontage` to 0, median, or mean. 
- **[ENG]** - Discard `GarageYrBlt`. There is no reasonable imputation value and it is correlated with `YearBuilt`, so the information is redundant.
- **[ENG]** - Impute `Electrical` to be the most frequent value.
- **[ENG]** - Change `MSSubClass` into a categorical feature. 
- **[ENG]** - Try: (1) impute MasVnrArea using the median/mean and MasVnrType with the most frequent observation, (2) first impute MasVnrType with the most frequent observation, then impute MasVnrArea with the average MasVnrArea for the most frequent MasVnrType.
- **[ENG]** - Drop the record where `Stone` has 0 `MasVnrArea` (id 1242). This might be erroneous record.
- **[ENG]** - Replace "NA" with "Absent" for some of the features.

We will perform the following steps primarily for linear regression. Although, we may experiment with using these steps for other models. 

- **[ENG]** - Discard `GarageYrBlt`
- **[ENG]** - Discard `TotRmsAbvGrd`
- **[ENG]** - Discard `TotalBsmtSF`
- **[ENG]** - Discard `GarageCars`
- **[ENG]** - Discard `2ndFlrSF`
- **[ENG]** - Fit to the logarithm of `SalePrice` to satisfy linearity.

We can try the following additional steps on the models, especially linear regression, to see if it improves performance. 

- **[ENG]** - Drop `SalePrice` outliers to see if that improves model performance.
- **[ENG]** - Try discarding `BsmtFullBath`
- **[ENG]** - Try discarding `TotRmsAbvGrd` and keeping the individual above-grade room counts. Compare this to just keeping `TotRmsAbvGrd `to determine if there is any performance boost.
- **[ENG]** - Try discarding `GrLivArea` or `1stFlrSF` or keeping both to see if there is any performance boost.
- **[ENG]** - Try discarding `HalfBath`. If we are preprocessing in an alternate way, where we discard `2ndFlrSF`, then try to keep `HalfBath`
- **[ENG]** - Try discarding either `BsmtUnfSF` or `BsmtFinSF1` or keeping both.
- **[ENG]** - Try discarding `BedroomAbvGr` over `GrLivArea`
- **[ENG]** - Consider dropping `FullBath` and `OverallQual`, which are correlated with `GrLivArea`. Note that `FullBath` is correlated with `OverallQual`
- **[ENG]** - Run experiments using `FullBath`, `YearRemodAdd` as features
- **[ENG]** - Run experiments using `MasVnrArea`, `Fireplaces`, `BsmtFinSF1`, `LotFrontage`, `2ndFlrSF`, `OpenPorchSF`, `WoodDeckSF` as features.
- **[ENG]** - Run experiments using features with absolute correlation $<=0.30$ as predictors.

If additional boosts in performance are necessary, then we might consider adding the categorical features. These features are somewhat undesirable  for linear regression, because they are not continuous features which would "play" better with the model. These might be useful in other models, however. 

- **[ENG]** - Empirically check if discarding `Neighborhood` or `MSZoning` improves performance
- **[ENG]** - Discard `BsmtCond` or `BsmtQual`, depending on which has less correlation with `SalePrice`
- **[ENG]** - Try discarding `BsmtExposure`, `BsmtFinType1`, and `BsmtFinType2`, as they are nominal variables, whereas `BsmtQual` and `BsmtCond` are continuous variables that capture some of the information. 
- **[ENG]** - Empirically check if discarding `GarageFinish` and/or `GarageType` in favour of `GarageQual` leads to performance boosts.
- **[ENG]** - Try manipulating the basement and garage categorical variables to remove redundancy and improve performance. Create a preprocessor that empirically checks every non-empty subset of the collinear features, then tests to see which one yields the best performance. 

Some additional suggestions for experiments:

- **[ENG]** - Try computing the number of years in-between YearBuilt and YearSold as a feature.



In [ ]:
X['MSSubClass']

In [ ]:
X['MSSubClass'] = X['MSSubClass'].astype('object')
X[cols_with_NA_category] = X[cols_with_NA_category].fillna(NA_category)

In [ ]:
# Recompute `X_train`, `X_valid` (using the same random seed, of course).
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=global_valid_size, random_state=global_random_state
)

# Recompute the numerical and object columns
mask = X.dtypes == 'object'
object_cols = list(mask[mask].index)
numerical_cols = list(set(X.columns) - set(object_cols))

In [ ]:
numerical_cols

In [ ]:
object_cols

In [ ]:
X_train

In [ ]:
X_valid